# Real-time Corrosion Detection System with RF-DETR
## Advanced Computer Vision Pipeline for R&D Material Science Monitoring

This notebook implements a comprehensive real-time computer vision system designed to help R&D researchers monitor corrosion patterns through intelligent behavior analysis. The system features:

🔧 **Core Pipeline:**
- RF-DETR (Real-time Fine-grained Detection Transformer) for corrosion pattern detection
- YOLOv11 for real-time object detection
- SAM2 for precise segmentation & refined bounding boxes

📊 **Dashboard Insights:**
- System performance (FPS, processing time, live trends)
- Corrosion severity ranking & progression stats
- Material health score, alerts, and corrosion type distribution
- Activity breakdown: Active Corrosion, Passive Corrosion, Pitting, Crevice Corrosion, Stress Corrosion

🧪 **Impact for R&D:**
- Early corrosion detection
- Automated 24/7 monitoring
- Reduced material failure
- Data-driven research and operational decisions

🧠 **Tech Stack:** Python · PyTorch · OpenCV · Ultralytics YOLO · SAM2 · Supervision · NumPy · Streamlit

In [ ]:
# Import essential libraries for the corrosion detection system
import os
import sys
import time
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet50
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# Install required packages (uncomment if needed)
# !pip install ultralytics
# !pip install segment-anything
# !pip install supervision
# !pip install streamlit
# !pip install plotly
# !pip install albumentations
# !pip install timm

try:
    from ultralytics import YOLO
    import supervision as sv
    print("✅ Ultralytics and Supervision imported successfully")
except ImportError:
    print("❌ Please install ultralytics and supervision: pip install ultralytics supervision")

try:
    import streamlit as st
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    print("✅ Streamlit and Plotly imported successfully")
except ImportError:
    print("❌ Please install streamlit and plotly: pip install streamlit plotly")

try:
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    print("✅ Albumentations imported successfully")
except ImportError:
    print("❌ Please install albumentations: pip install albumentations")

In [ ]:
# Dataset Configuration and Path Setup
import os
from pathlib import Path

# Configure dataset paths
DATASET_ROOT = Path(r"C:\Users\gopeami\OneDrive - Vesuvius\Desktop\PhD13- 2025-2026\ML Practice\Material property Design\Corrosion detection\Dataset")
IMAGES_DIR = DATASET_ROOT / "images"
ANNOTATIONS_DIR = DATASET_ROOT / "annotations"
TRAIN_DIR = DATASET_ROOT / "train"
VAL_DIR = DATASET_ROOT / "validation" 
TEST_DIR = DATASET_ROOT / "test"

# Create directories if they don't exist
for directory in [DATASET_ROOT, IMAGES_DIR, ANNOTATIONS_DIR, TRAIN_DIR, VAL_DIR, TEST_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print("📁 Dataset Configuration:")
print(f"  • Dataset Root: {DATASET_ROOT}")
print(f"  • Images Directory: {IMAGES_DIR}")
print(f"  • Annotations Directory: {ANNOTATIONS_DIR}")
print(f"  • Training Directory: {TRAIN_DIR}")
print(f"  • Validation Directory: {VAL_DIR}")
print(f"  • Test Directory: {TEST_DIR}")

# Check if dataset exists
if DATASET_ROOT.exists():
    print("✅ Dataset directory exists")
    
    # List subdirectories and files
    subdirs = [d for d in DATASET_ROOT.iterdir() if d.is_dir()]
    files = [f for f in DATASET_ROOT.iterdir() if f.is_file()]
    
    print(f"  • Subdirectories: {len(subdirs)}")
    for subdir in subdirs[:5]:  # Show first 5
        print(f"    - {subdir.name}")
    if len(subdirs) > 5:
        print(f"    ... and {len(subdirs) - 5} more")
    
    print(f"  • Files: {len(files)}")
    for file in files[:5]:  # Show first 5
        print(f"    - {file.name}")
    if len(files) > 5:
        print(f"    ... and {len(files) - 5} more")
        
else:
    print("⚠️ Dataset directory does not exist")
    print("   Please ensure the dataset is available at the specified path")

# Dataset structure expected:
"""
Dataset/
├── train/
│   ├── images/
│   │   ├── img_001.jpg
│   │   ├── img_002.jpg
│   │   └── ...
│   └── labels/
│       ├── img_001.txt (YOLO format)
│       ├── img_002.txt
│       └── ...
├── validation/
│   ├── images/
│   └── labels/
├── test/
│   ├── images/
│   └── labels/
├── classes.names
└── dataset.yaml
"""

In [ ]:
# Dataset Loading and Management Functions
class CorrosionDatasetLoader:
    """Dataset loader for corrosion detection data"""
    
    def __init__(self, dataset_root=DATASET_ROOT):
        self.dataset_root = Path(dataset_root)
        self.train_dir = self.dataset_root / "train"
        self.val_dir = self.dataset_root / "validation" 
        self.test_dir = self.dataset_root / "test"
        
        # Class mapping for corrosion types
        self.class_names = [
            'uniform_corrosion',
            'pitting_corrosion', 
            'crevice_corrosion',
            'stress_corrosion',
            'galvanic_corrosion',
            'erosion_corrosion'
        ]
        
    def scan_dataset(self):
        """Scan dataset and return statistics"""
        stats = {
            'train': self._scan_split('train'),
            'validation': self._scan_split('validation'),
            'test': self._scan_split('test')
        }
        
        return stats
    
    def _scan_split(self, split_name):
        """Scan a specific data split"""
        split_dir = self.dataset_root / split_name
        images_dir = split_dir / "images"
        labels_dir = split_dir / "labels"
        
        stats = {
            'images_count': 0,
            'labels_count': 0,
            'image_formats': {},
            'missing_labels': [],
            'missing_images': []
        }
        
        if not split_dir.exists():
            return stats
            
        # Count images
        if images_dir.exists():
            image_files = []
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']:
                image_files.extend(images_dir.glob(ext))
            
            stats['images_count'] = len(image_files)
            
            # Count formats
            for img_file in image_files:
                ext = img_file.suffix.lower()
                stats['image_formats'][ext] = stats['image_formats'].get(ext, 0) + 1
        
        # Count labels
        if labels_dir.exists():
            label_files = list(labels_dir.glob('*.txt'))
            stats['labels_count'] = len(label_files)
            
            # Check for missing correspondences
            if images_dir.exists():
                for img_file in image_files:
                    label_file = labels_dir / f"{img_file.stem}.txt"
                    if not label_file.exists():
                        stats['missing_labels'].append(img_file.name)
                
                for label_file in label_files:
                    # Check if corresponding image exists
                    img_exists = False
                    for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
                        img_file = images_dir / f"{label_file.stem}{ext}"
                        if img_file.exists():
                            img_exists = True
                            break
                    if not img_exists:
                        stats['missing_images'].append(label_file.name)
        
        return stats
    
    def create_yolo_yaml(self, output_path=None):
        """Create YOLO dataset configuration file"""
        if output_path is None:
            output_path = self.dataset_root / "dataset.yaml"
        
        yaml_content = f"""# Corrosion Detection Dataset Configuration
# Dataset root path
path: {str(self.dataset_root).replace(os.sep, '/')}

# Train, validation, and test sets
train: train/images
val: validation/images
test: test/images

# Number of classes
nc: {len(self.class_names)}

# Class names
names: {self.class_names}

# Class descriptions
descriptions:
  0: "Uniform corrosion - widespread surface corrosion"
  1: "Pitting corrosion - localized holes in metal surface"
  2: "Crevice corrosion - corrosion in confined spaces"
  3: "Stress corrosion - cracking due to stress and environment"
  4: "Galvanic corrosion - corrosion due to different metals"
  5: "Erosion corrosion - corrosion enhanced by fluid flow"
"""
        
        with open(output_path, 'w') as f:
            f.write(yaml_content)
        
        print(f"✅ YOLO dataset configuration created: {output_path}")
        return output_path
    
    def load_image_paths(self, split='train'):
        """Load image paths for a specific split"""
        split_dir = self.dataset_root / split / "images"
        
        if not split_dir.exists():
            print(f"❌ {split} images directory does not exist: {split_dir}")
            return []
        
        image_paths = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']:
            image_paths.extend(split_dir.glob(ext))
        
        return [str(path) for path in image_paths]
    
    def load_annotations(self, split='train'):
        """Load annotations for a specific split"""
        labels_dir = self.dataset_root / split / "labels"
        
        if not labels_dir.exists():
            print(f"❌ {split} labels directory does not exist: {labels_dir}")
            return []
        
        annotations = []
        label_files = list(labels_dir.glob('*.txt'))
        
        for label_file in label_files:
            annotation = self._parse_yolo_annotation(label_file)
            if annotation:
                annotations.append(annotation)
        
        return annotations
    
    def _parse_yolo_annotation(self, label_file):
        """Parse YOLO format annotation file"""
        try:
            with open(label_file, 'r') as f:
                lines = f.readlines()
            
            boxes = []
            labels = []
            
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center = float(parts[1])
                    y_center = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                    
                    # Convert YOLO format (center, width, height) to (x1, y1, x2, y2)
                    x1 = x_center - width / 2
                    y1 = y_center - height / 2
                    x2 = x_center + width / 2
                    y2 = y_center + height / 2
                    
                    boxes.append([x1, y1, x2, y2])
                    labels.append(class_id)
            
            return {
                'file': str(label_file),
                'boxes': boxes,
                'labels': labels
            }
            
        except Exception as e:
            print(f"❌ Error parsing {label_file}: {e}")
            return None
    
    def print_dataset_summary(self):
        """Print comprehensive dataset summary"""
        print("\\n" + "="*60)
        print("CORROSION DATASET SUMMARY")
        print("="*60)
        
        stats = self.scan_dataset()
        
        total_images = sum(split['images_count'] for split in stats.values())
        total_labels = sum(split['labels_count'] for split in stats.values())
        
        print(f"\\nDataset Root: {self.dataset_root}")
        print(f"Total Images: {total_images}")
        print(f"Total Labels: {total_labels}")
        
        for split_name, split_stats in stats.items():
            print(f"\\n📊 {split_name.upper()} SET:")
            print(f"  • Images: {split_stats['images_count']}")
            print(f"  • Labels: {split_stats['labels_count']}")
            
            if split_stats['image_formats']:
                print("  • Image formats:")
                for fmt, count in split_stats['image_formats'].items():
                    print(f"    - {fmt}: {count}")
            
            if split_stats['missing_labels']:
                print(f"  ⚠️ Missing labels: {len(split_stats['missing_labels'])}")
                if len(split_stats['missing_labels']) <= 5:
                    for missing in split_stats['missing_labels']:
                        print(f"    - {missing}")
                else:
                    for missing in split_stats['missing_labels'][:3]:
                        print(f"    - {missing}")
                    print(f"    ... and {len(split_stats['missing_labels']) - 3} more")
            
            if split_stats['missing_images']:
                print(f"  ⚠️ Missing images: {len(split_stats['missing_images'])}")
                if len(split_stats['missing_images']) <= 5:
                    for missing in split_stats['missing_images']:
                        print(f"    - {missing}")
                else:
                    for missing in split_stats['missing_images'][:3]:
                        print(f"    - {missing}")
                    print(f"    ... and {len(split_stats['missing_images']) - 3} more")
        
        print("\\n" + "="*60)

# Initialize dataset loader and scan the dataset
dataset_loader = CorrosionDatasetLoader()
dataset_loader.print_dataset_summary()

# Create YOLO configuration file
try:
    yaml_path = dataset_loader.create_yolo_yaml()
except Exception as e:
    print(f"⚠️ Could not create YOLO config: {e}")

print(f"\\n🔍 Dataset loader initialized and ready to use!")
print(f"📝 Use 'dataset_loader.load_image_paths(\"train\")' to get training image paths")
print(f"📝 Use 'dataset_loader.load_annotations(\"train\")' to get training annotations")

In [ ]:
# RF-DETR: Real-time Fine-grained Detection Transformer Implementation
class PositionalEncoding(nn.Module):
    """Positional encoding for transformer architecture"""
    def __init__(self, d_model=256, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class MultiHeadAttention(nn.Module):
    """Multi-head attention mechanism for RF-DETR"""
    def __init__(self, d_model=256, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)
        
    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)
        seq_len = q.size(1)
        
        # Linear transformations
        Q = self.query(q).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(k).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(v).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Scaled dot-product attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attention = F.softmax(scores, dim=-1)
        context = torch.matmul(attention, V)
        
        # Concatenate heads
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        output = self.out(context)
        
        return output, attention

In [ ]:
# Transformer Block for RF-DETR
class TransformerBlock(nn.Module):
    """Transformer block with multi-head attention and feed-forward network"""
    def __init__(self, d_model=256, num_heads=8, d_ff=1024, dropout=0.1):
        super(TransformerBlock, self).__init__()
        
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Multi-head attention with residual connection
        attn_output, attention_weights = self.attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Feed-forward with residual connection
        ff_output = self.feed_forward(x)
        x = self.norm2(x + ff_output)
        
        return x, attention_weights

# Backbone feature extractor for RF-DETR
class CorrosionBackbone(nn.Module):
    """Feature extraction backbone using ResNet50"""
    def __init__(self, pretrained=True):
        super(CorrosionBackbone, self).__init__()
        
        # Load pretrained ResNet50
        resnet = resnet50(pretrained=pretrained)
        
        # Remove final layers
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])
        
        # Feature projection layer
        self.proj = nn.Conv2d(2048, 256, kernel_size=1)
        
    def forward(self, x):
        features = self.backbone(x)  # [B, 2048, H/32, W/32]
        features = self.proj(features)  # [B, 256, H/32, W/32]
        
        # Flatten spatial dimensions for transformer
        B, C, H, W = features.shape
        features = features.flatten(2).transpose(1, 2)  # [B, H*W, 256]
        
        return features, (H, W)

In [ ]:
# RF-DETR Main Architecture
class RFDETR(nn.Module):
    """
    Real-time Fine-grained Detection Transformer for Corrosion Detection
    """
    def __init__(self, num_classes=6, num_queries=100, d_model=256, num_heads=8, 
                 num_encoder_layers=6, num_decoder_layers=6):
        super(RFDETR, self).__init__()
        
        self.num_classes = num_classes
        self.num_queries = num_queries
        self.d_model = d_model
        
        # Backbone feature extractor
        self.backbone = CorrosionBackbone()
        
        # Positional encoding
        self.pos_encoding = PositionalEncoding(d_model)
        
        # Transformer Encoder
        self.encoder_layers = nn.ModuleList([
            TransformerBlock(d_model, num_heads) for _ in range(num_encoder_layers)
        ])
        
        # Object queries (learnable embeddings)
        self.object_queries = nn.Embedding(num_queries, d_model)
        
        # Transformer Decoder
        self.decoder_layers = nn.ModuleList([
            TransformerBlock(d_model, num_heads) for _ in range(num_decoder_layers)
        ])
        
        # Prediction heads
        self.class_embed = nn.Linear(d_model, num_classes + 1)  # +1 for background
        self.bbox_embed = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, 4)  # x, y, w, h
        )
        
        # Corrosion severity prediction head
        self.severity_embed = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, 5)  # 5 severity levels
        )
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # Extract features from backbone
        features, (H, W) = self.backbone(x)  # [B, H*W, 256]
        
        # Add positional encoding
        features = self.pos_encoding(features)
        
        # Transformer encoder
        encoder_output = features
        for encoder_layer in self.encoder_layers:
            encoder_output, _ = encoder_layer(encoder_output)
        
        # Object queries for decoder
        queries = self.object_queries.weight.unsqueeze(0).repeat(batch_size, 1, 1)  # [B, num_queries, 256]
        
        # Transformer decoder
        decoder_output = queries
        for decoder_layer in self.decoder_layers:
            decoder_output, _ = decoder_layer(decoder_output)
        
        # Prediction heads
        class_logits = self.class_embed(decoder_output)  # [B, num_queries, num_classes+1]
        bbox_coords = self.bbox_embed(decoder_output)    # [B, num_queries, 4]
        bbox_coords = torch.sigmoid(bbox_coords)         # Normalize to [0, 1]
        severity_logits = self.severity_embed(decoder_output)  # [B, num_queries, 5]
        
        return {
            'class_logits': class_logits,
            'bbox_coords': bbox_coords,
            'severity_logits': severity_logits,
            'features': encoder_output
        }

In [ ]:
# Loss Functions for RF-DETR Training
class RFDETRLoss(nn.Module):
    """Custom loss function for RF-DETR training"""
    
    def __init__(self, num_classes, cost_class=1.0, cost_bbox=5.0, cost_giou=2.0, cost_severity=1.0):
        super(RFDETRLoss, self).__init__()
        self.num_classes = num_classes
        self.cost_class = cost_class
        self.cost_bbox = cost_bbox
        self.cost_giou = cost_giou
        self.cost_severity = cost_severity
        
        # Loss functions
        self.ce_loss = nn.CrossEntropyLoss()
        self.l1_loss = nn.L1Loss(reduction='none')
        self.severity_loss = nn.CrossEntropyLoss()
        
    def box_iou(self, boxes1, boxes2):
        """Calculate IoU between two sets of boxes"""
        # Convert from (cx, cy, w, h) to (x1, y1, x2, y2)
        boxes1_corners = self.box_cxcywh_to_xyxy(boxes1)
        boxes2_corners = self.box_cxcywh_to_xyxy(boxes2)
        
        # Calculate intersection
        lt = torch.max(boxes1_corners[:, None, :2], boxes2_corners[:, :2])
        rb = torch.min(boxes1_corners[:, None, 2:], boxes2_corners[:, 2:])
        
        wh = (rb - lt).clamp(min=0)
        inter = wh[:, :, 0] * wh[:, :, 1]
        
        # Calculate areas
        area1 = (boxes1_corners[:, 2] - boxes1_corners[:, 0]) * (boxes1_corners[:, 3] - boxes1_corners[:, 1])
        area2 = (boxes2_corners[:, 2] - boxes2_corners[:, 0]) * (boxes2_corners[:, 3] - boxes2_corners[:, 1])
        
        # Calculate union and IoU
        union = area1[:, None] + area2 - inter
        iou = inter / union
        
        return iou
    
    def box_cxcywh_to_xyxy(self, boxes):
        """Convert center format to corner format"""
        cx, cy, w, h = boxes.unbind(-1)
        x1 = cx - 0.5 * w
        y1 = cy - 0.5 * h
        x2 = cx + 0.5 * w
        y2 = cy + 0.5 * h
        return torch.stack([x1, y1, x2, y2], dim=-1)
    
    def generalized_box_iou(self, boxes1, boxes2):
        """Calculate Generalized IoU"""
        iou = self.box_iou(boxes1, boxes2)
        
        boxes1_corners = self.box_cxcywh_to_xyxy(boxes1)
        boxes2_corners = self.box_cxcywh_to_xyxy(boxes2)
        
        # Calculate enclosing box
        lt = torch.min(boxes1_corners[:, None, :2], boxes2_corners[:, :2])
        rb = torch.max(boxes1_corners[:, None, 2:], boxes2_corners[:, 2:])
        
        wh = (rb - lt).clamp(min=0)
        area_c = wh[:, :, 0] * wh[:, :, 1]
        
        # Calculate areas
        area1 = (boxes1_corners[:, 2] - boxes1_corners[:, 0]) * (boxes1_corners[:, 3] - boxes1_corners[:, 1])
        area2 = (boxes2_corners[:, 2] - boxes2_corners[:, 0]) * (boxes2_corners[:, 3] - boxes2_corners[:, 1])
        union = area1[:, None] + area2 - iou * area1[:, None]
        
        # Generalized IoU
        giou = iou - (area_c - union) / area_c
        
        return giou

In [ ]:
# Updated Corrosion Dataset Class with Configured Paths
class CorrosionDataset(Dataset):
    """Dataset class for corrosion detection training with configured paths"""
    
    def __init__(self, split='train', transforms=None, is_train=True, dataset_root=None):
        if dataset_root is None:
            dataset_root = DATASET_ROOT
        
        self.dataset_root = Path(dataset_root)
        self.split = split
        self.transforms = transforms
        self.is_train = is_train
        
        # Load data using the dataset loader
        loader = CorrosionDatasetLoader(dataset_root)
        self.image_paths = loader.load_image_paths(split)
        self.annotations = loader.load_annotations(split)
        
        # Ensure image paths and annotations match
        self._validate_data()
        
        # Corrosion type mapping
        self.corrosion_types = {
            0: 'uniform_corrosion',
            1: 'pitting_corrosion', 
            2: 'crevice_corrosion',
            3: 'stress_corrosion',
            4: 'galvanic_corrosion',
            5: 'erosion_corrosion'
        }
        
        # Severity levels
        self.severity_levels = {
            0: 'none',
            1: 'mild',
            2: 'moderate', 
            3: 'severe',
            4: 'critical'
        }
        
        print(f"📊 {split.upper()} dataset: {len(self.image_paths)} images, {len(self.annotations)} annotations")
    
    def _validate_data(self):
        """Validate that images and annotations match"""
        if len(self.image_paths) != len(self.annotations):
            print(f"⚠️ Warning: {len(self.image_paths)} images but {len(self.annotations)} annotations")
            
            # Try to match by filename
            image_stems = [Path(img_path).stem for img_path in self.image_paths]
            matched_annotations = []
            
            for img_path in self.image_paths:
                img_stem = Path(img_path).stem
                # Find corresponding annotation
                matching_annotation = None
                for ann in self.annotations:
                    ann_stem = Path(ann['file']).stem
                    if ann_stem == img_stem:
                        matching_annotation = ann
                        break
                
                if matching_annotation:
                    matched_annotations.append(matching_annotation)
                else:
                    # Create empty annotation
                    matched_annotations.append({
                        'file': img_path,
                        'boxes': [],
                        'labels': []
                    })
            
            self.annotations = matched_annotations
            print(f"✅ Matched {len(self.annotations)} annotations to images")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        image_path = self.image_paths[idx]
        
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Could not load image: {image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"❌ Error loading image {image_path}: {e}")
            # Return a dummy image
            image = np.zeros((512, 512, 3), dtype=np.uint8)
        
        # Get annotations
        annotation = self.annotations[idx]
        
        # Apply transforms
        if self.transforms:
            if self.is_train and len(annotation['boxes']) > 0:
                # Training augmentations with bounding boxes
                try:
                    transformed = self.transforms(
                        image=image,
                        bboxes=annotation['boxes'],
                        category_ids=annotation['labels']
                    )
                    image = transformed['image']
                    boxes = transformed['bboxes']
                    labels = transformed['category_ids']
                except Exception as e:
                    print(f"⚠️ Transform error for {image_path}: {e}")
                    # Fallback to original data
                    boxes = annotation['boxes']
                    labels = annotation['labels']
                    if len(image.shape) == 3:
                        image = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
                    else:
                        image = torch.from_numpy(image).float() / 255.0
            else:
                # Validation transforms without bbox augmentation
                try:
                    if hasattr(self.transforms, 'transforms'):
                        # Handle albumentations compose
                        transformed = self.transforms(image=image)
                        image = transformed['image']
                    else:
                        # Handle torchvision transforms
                        image = self.transforms(image)
                    boxes = annotation['boxes']
                    labels = annotation['labels']
                except Exception as e:
                    print(f"⚠️ Transform error for {image_path}: {e}")
                    boxes = annotation['boxes']
                    labels = annotation['labels']
                    if len(image.shape) == 3:
                        image = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
                    else:
                        image = torch.from_numpy(image).float() / 255.0
        else:
            boxes = annotation['boxes']
            labels = annotation['labels']
            # Normalize image
            image = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
        
        # Convert to tensors
        if len(boxes) > 0:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.long)
            # Generate random severity for demo (in real scenario, this would come from annotations)
            severity = torch.tensor([np.random.randint(1, 5) for _ in labels], dtype=torch.long)
        else:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.long)
            severity = torch.zeros((0,), dtype=torch.long)
        
        target = {
            'boxes': boxes,
            'labels': labels,
            'severity': severity,
            'image_id': idx,
            'image_path': image_path
        }
        
        return image, target

# Updated data loading functions using configured paths
def create_corrosion_dataloaders(batch_size=4, num_workers=2, dataset_root=None):
    """Create train and validation dataloaders from configured dataset"""
    
    if dataset_root is None:
        dataset_root = DATASET_ROOT
    
    print(f"🔄 Creating dataloaders from {dataset_root}")
    
    # Create datasets
    train_dataset = CorrosionDataset(
        split='train',
        transforms=get_train_transforms(),
        is_train=True,
        dataset_root=dataset_root
    )
    
    val_dataset = CorrosionDataset(
        split='validation',
        transforms=get_val_transforms(),
        is_train=False,
        dataset_root=dataset_root
    )
    
    # Create dataloaders
    def collate_fn(batch):
        """Custom collate function for variable-sized annotations"""
        images = []
        targets = []
        
        for image, target in batch:
            images.append(image)
            targets.append(target)
        
        # Stack images
        try:
            images = torch.stack(images, 0)
        except Exception as e:
            print(f"Error stacking images: {e}")
            # Fallback: pad to same size
            max_h = max(img.shape[1] for img in images)
            max_w = max(img.shape[2] for img in images)
            
            padded_images = []
            for img in images:
                h, w = img.shape[1], img.shape[2]
                padded = torch.zeros(3, max_h, max_w)
                padded[:, :h, :w] = img
                padded_images.append(padded)
            images = torch.stack(padded_images, 0)
        
        return images, targets
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=torch.cuda.is_available()
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=torch.cuda.is_available()
    )
    
    print(f"✅ Created dataloaders:")
    print(f"  • Train: {len(train_dataset)} samples, {len(train_loader)} batches")
    print(f"  • Validation: {len(val_dataset)} samples, {len(val_loader)} batches")
    
    return train_loader, val_loader

# Updated data augmentation transforms with better error handling
def get_train_transforms():
    """Training data augmentations with error handling"""
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(p=0.5, brightness_limit=0.2, contrast_limit=0.2),
        A.HueSaturationValue(p=0.3, hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10),
        A.GaussNoise(p=0.3, var_limit=(0, 50)),
        A.Blur(blur_limit=3, p=0.3),
        A.CLAHE(p=0.3),
        A.Resize(512, 512, always_apply=True),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], always_apply=True),
        ToTensorV2(always_apply=True)
    ], bbox_params=A.BboxParams(
        format='albumentations', 
        label_fields=['category_ids'],
        min_visibility=0.1,
        min_area=100
    ))

def get_val_transforms():
    """Validation data transforms with error handling"""
    return A.Compose([
        A.Resize(512, 512, always_apply=True),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], always_apply=True),
        ToTensorV2(always_apply=True)
    ])

# Test dataset loading with configured paths
try:
    print("\\n🧪 Testing dataset loading...")
    train_loader, val_loader = create_corrosion_dataloaders(batch_size=2, num_workers=0)
    
    # Test loading one batch
    try:
        train_batch = next(iter(train_loader))
        images, targets = train_batch
        print(f"✅ Successfully loaded training batch:")
        print(f"  • Images shape: {images.shape}")
        print(f"  • Number of targets: {len(targets)}")
        
        # Show target info
        for i, target in enumerate(targets[:2]):  # Show first 2
            print(f"  • Sample {i}: {target['boxes'].shape[0]} boxes, labels: {target['labels'].tolist()}")
            
    except Exception as e:
        print(f"⚠️ Error loading training batch: {e}")
        print("   This might be expected if no dataset is available yet")
        
except Exception as e:
    print(f"⚠️ Could not create dataloaders: {e}")
    print("   This is expected if the dataset directory is empty")

In [ ]:
# Updated YOLOv11 Integration with Configured Dataset Paths
class YOLOCorrosionDetector:
    """YOLOv11-based corrosion detector with configured dataset paths"""
    
    def __init__(self, model_path=None, conf_threshold=0.5, iou_threshold=0.4, dataset_root=None):
        self.conf_threshold = conf_threshold
        self.iou_threshold = iou_threshold
        
        # Set dataset root
        if dataset_root is None:
            self.dataset_root = DATASET_ROOT
        else:
            self.dataset_root = Path(dataset_root)
        
        # Initialize YOLOv11 model
        if model_path and os.path.exists(model_path):
            self.model = YOLO(model_path)
            print(f"✅ Loaded custom YOLO model: {model_path}")
        else:
            # Use pretrained YOLOv11 and fine-tune for corrosion
            self.model = YOLO('yolo11n.pt')  # Start with nano version for speed
            print("✅ Loaded pretrained YOLOv11n model")
        
        # Corrosion class names
        self.class_names = [
            'uniform_corrosion', 'pitting_corrosion', 'crevice_corrosion',
            'stress_corrosion', 'galvanic_corrosion', 'erosion_corrosion'
        ]
        
        # Color palette for visualization
        self.colors = [
            (255, 0, 0),    # Red - Uniform
            (0, 255, 0),    # Green - Pitting  
            (0, 0, 255),    # Blue - Crevice
            (255, 255, 0),  # Yellow - Stress
            (255, 0, 255),  # Magenta - Galvanic
            (0, 255, 255)   # Cyan - Erosion
        ]
        
        print(f"🎯 Configured for corrosion detection with {len(self.class_names)} classes")
        print(f"📁 Dataset root: {self.dataset_root}")
    
    def prepare_dataset_for_yolo(self):
        """Prepare dataset structure and YAML file for YOLO training"""
        
        print("📋 Preparing dataset for YOLO training...")
        
        # Create YAML configuration
        loader = CorrosionDatasetLoader(self.dataset_root)
        yaml_path = loader.create_yolo_yaml()
        
        # Validate dataset structure
        stats = loader.scan_dataset()
        
        print("📊 Dataset validation:")
        for split, split_stats in stats.items():
            if split_stats['images_count'] > 0:
                print(f"  ✅ {split}: {split_stats['images_count']} images, {split_stats['labels_count']} labels")
            else:
                print(f"  ⚠️ {split}: No data found")
        
        # Check if dataset is ready for training
        train_ready = stats['train']['images_count'] > 0 and stats['train']['labels_count'] > 0
        val_ready = stats['validation']['images_count'] > 0 and stats['validation']['labels_count'] > 0
        
        if train_ready and val_ready:
            print("✅ Dataset is ready for YOLO training!")
            return str(yaml_path)
        else:
            print("❌ Dataset is not ready for training:")
            if not train_ready:
                print("   - Missing training data")
            if not val_ready:
                print("   - Missing validation data")
            return None
    
    def train_model(self, epochs=100, batch_size=16, image_size=640, device_id=None):
        """Train YOLOv11 model on corrosion dataset"""
        
        # Prepare dataset
        yaml_path = self.prepare_dataset_for_yolo()
        if yaml_path is None:
            print("❌ Cannot train: Dataset not ready")
            return None
        
        print(f"🚀 Starting YOLO training...")
        print(f"  • Dataset config: {yaml_path}")
        print(f"  • Epochs: {epochs}")
        print(f"  • Batch size: {batch_size}")
        print(f"  • Image size: {image_size}")
        print(f"  • Device: {device if device_id is None else device_id}")
        
        try:
            # Configure training parameters
            results = self.model.train(
                data=yaml_path,
                epochs=epochs,
                batch=batch_size,
                imgsz=image_size,
                device=device if device_id is None else device_id,
                project=str(self.dataset_root.parent / 'runs' / 'corrosion_detection'),
                name=f'yolo11_corrosion_{int(time.time())}',
                save=True,
                plots=True,
                verbose=True,
                patience=50,  # Early stopping patience
                save_period=10,  # Save checkpoint every 10 epochs
                workers=4,  # Number of worker threads
                cos_lr=True,  # Use cosine learning rate schedule
                close_mosaic=10,  # Close mosaic augmentation in last 10 epochs
                amp=True,  # Use automatic mixed precision
            )
            
            print("✅ Training completed successfully!")
            
            # Update model to the best trained weights
            best_model_path = results.save_dir / 'weights' / 'best.pt'
            if best_model_path.exists():
                self.model = YOLO(str(best_model_path))
                print(f"✅ Model updated to best weights: {best_model_path}")
            
            return results
            
        except Exception as e:
            print(f"❌ Training failed: {e}")
            return None
    
    def predict(self, image, return_annotated=True, save_results=False):
        """Predict corrosion in image with enhanced error handling"""
        
        try:
            # Ensure image is in correct format
            if isinstance(image, str):
                # Load image from path
                image = cv2.imread(image)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            elif len(image.shape) == 4:
                # Remove batch dimension if present
                image = image[0]
            
            # Run inference
            results = self.model(
                image,
                conf=self.conf_threshold,
                iou=self.iou_threshold,
                verbose=False
            )
            
            detections = []
            annotated_image = image.copy() if return_annotated else None
            
            for result in results:
                boxes = result.boxes
                if boxes is not None and len(boxes) > 0:
                    for box in boxes:
                        # Extract detection info
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        conf = box.conf[0].cpu().numpy()
                        cls = int(box.cls[0].cpu().numpy())
                        
                        # Validate class index
                        if cls >= len(self.class_names):
                            print(f"⚠️ Invalid class index: {cls}")
                            continue
                        
                        detection = {
                            'bbox': [float(x1), float(y1), float(x2), float(y2)],
                            'confidence': float(conf),
                            'class': cls,
                            'class_name': self.class_names[cls],
                            'area': float((x2 - x1) * (y2 - y1))
                        }
                        detections.append(detection)
                        
                        if return_annotated:
                            # Draw bounding box
                            color = self.colors[cls % len(self.colors)]
                            cv2.rectangle(annotated_image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                            
                            # Draw label
                            label = f"{detection['class_name'].replace('_', ' ').title()}: {conf:.2f}"
                            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                            
                            # Background for label
                            cv2.rectangle(annotated_image, 
                                        (int(x1), int(y1) - label_size[1] - 10), 
                                        (int(x1) + label_size[0], int(y1)), 
                                        color, -1)
                            
                            # Label text
                            cv2.putText(annotated_image, label, (int(x1), int(y1) - 5), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # Save results if requested
            if save_results and detections:
                timestamp = int(time.time())
                results_dir = self.dataset_root.parent / 'results' / f'detection_{timestamp}'
                results_dir.mkdir(parents=True, exist_ok=True)
                
                if return_annotated:
                    cv2.imwrite(str(results_dir / 'annotated.jpg'), 
                               cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
                
                # Save detection data
                import json
                with open(results_dir / 'detections.json', 'w') as f:
                    json.dump(detections, f, indent=2)
                
                print(f"💾 Results saved to: {results_dir}")
            
            return detections, annotated_image if return_annotated else None
            
        except Exception as e:
            print(f"❌ Prediction error: {e}")
            return [], image if return_annotated else None
    
    def evaluate_on_test_set(self):
        """Evaluate model on test set if available"""
        
        loader = CorrosionDatasetLoader(self.dataset_root)
        stats = loader.scan_dataset()
        
        if stats['test']['images_count'] == 0:
            print("⚠️ No test set available for evaluation")
            return None
        
        print(f"🧪 Evaluating on test set ({stats['test']['images_count']} images)...")
        
        try:
            # Run validation on test set
            test_images_dir = self.dataset_root / 'test' / 'images'
            
            results = self.model.val(
                data=str(self.dataset_root / 'dataset.yaml'),
                split='test',
                save=True,
                verbose=True
            )
            
            print("✅ Evaluation completed!")
            print(f"  • mAP50: {results.box.map50:.3f}")
            print(f"  • mAP50-95: {results.box.map:.3f}")
            
            return results
            
        except Exception as e:
            print(f"❌ Evaluation failed: {e}")
            return None

# Initialize YOLO detector with configured dataset
print("🤖 Initializing YOLO Corrosion Detector...")
yolo_detector = YOLOCorrosionDetector(dataset_root=DATASET_ROOT)

# Test with a sample if dataset exists
try:
    loader = CorrosionDatasetLoader()
    train_images = loader.load_image_paths('train')
    
    if train_images:
        print(f"\\n🔍 Testing detection on sample image...")
        sample_image = train_images[0]
        
        if os.path.exists(sample_image):
            detections, annotated = yolo_detector.predict(sample_image, return_annotated=True)
            print(f"✅ Detection test completed: {len(detections)} detections found")
        else:
            print("⚠️ Sample image path does not exist")
    else:
        print("ℹ️ No training images found for testing")
        
except Exception as e:
    print(f"ℹ️ Could not test detection: {e}")

print("\\n💡 Ready for training! Use:")
print("   yolo_detector.train_model(epochs=50, batch_size=8)")
print("   yolo_detector.predict(image_path)")

In [ ]:
# Training Workflow and Dataset Management
class CorrosionTrainingWorkflow:
    """Complete training workflow for corrosion detection system"""
    
    def __init__(self, dataset_root=None):
        self.dataset_root = Path(dataset_root) if dataset_root else DATASET_ROOT
        self.yolo_detector = None
        self.rfdetr_model = None
        
        print(f"🚀 Corrosion Training Workflow initialized")
        print(f"📁 Dataset root: {self.dataset_root}")
    
    def setup_training_environment(self):
        """Setup complete training environment"""
        
        print("⚙️ Setting up training environment...")
        
        # 1. Validate dataset
        loader = CorrosionDatasetLoader(self.dataset_root)
        stats = loader.scan_dataset()
        
        print("\\n📊 Dataset Status:")
        for split, split_stats in stats.items():
            status = "✅" if split_stats['images_count'] > 0 else "❌"
            print(f"  {status} {split.title()}: {split_stats['images_count']} images, {split_stats['labels_count']} labels")
        
        # 2. Create YOLO configuration
        try:
            yaml_path = loader.create_yolo_yaml()
            print(f"✅ YOLO config created: {yaml_path}")
        except Exception as e:
            print(f"❌ Failed to create YOLO config: {e}")
        
        # 3. Initialize models
        try:
            self.yolo_detector = YOLOCorrosionDetector(dataset_root=self.dataset_root)
            print("✅ YOLO detector initialized")
        except Exception as e:
            print(f"❌ YOLO initialization failed: {e}")
        
        try:
            self.rfdetr_model = RFDETR(num_classes=6, num_queries=100)
            print(f"✅ RF-DETR initialized ({sum(p.numel() for p in self.rfdetr_model.parameters()):,} parameters)")
        except Exception as e:
            print(f"❌ RF-DETR initialization failed: {e}")
        
        # 4. Check GPU availability
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            print(f"✅ GPU available: {gpu_name} ({gpu_memory:.1f}GB)")
        else:
            print("⚠️ No GPU available - training will be slower")
        
        return stats
    
    def start_yolo_training(self, epochs=100, batch_size=16, image_size=640):
        """Start YOLO training with optimal parameters"""
        
        if self.yolo_detector is None:
            print("❌ YOLO detector not initialized")
            return None
        
        print(f"🚀 Starting YOLO training workflow...")
        
        # Optimize batch size based on available memory
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            if gpu_memory < 8:
                batch_size = min(batch_size, 8)
                print(f"⚡ Adjusted batch size to {batch_size} for GPU memory")
        else:
            batch_size = min(batch_size, 4)
            print(f"⚡ Adjusted batch size to {batch_size} for CPU training")
        
        # Start training
        results = self.yolo_detector.train_model(
            epochs=epochs,
            batch_size=batch_size,
            image_size=image_size
        )
        
        if results:
            print("✅ YOLO training completed successfully!")
            return results
        else:
            print("❌ YOLO training failed")
            return None
    
    def start_rfdetr_training(self, epochs=50, batch_size=4):
        """Start RF-DETR training"""
        
        if self.rfdetr_model is None:
            print("❌ RF-DETR model not initialized")
            return None
        
        print(f"🚀 Starting RF-DETR training workflow...")
        
        try:
            # Create dataloaders
            train_loader, val_loader = create_corrosion_dataloaders(
                batch_size=batch_size, 
                num_workers=0,  # Set to 0 to avoid multiprocessing issues
                dataset_root=self.dataset_root
            )
            
            # Initialize trainer
            trainer = RFDETRTrainer(
                self.rfdetr_model, 
                train_loader, 
                val_loader, 
                num_epochs=epochs
            )
            
            # Start training
            train_losses, val_losses = trainer.train()
            
            print("✅ RF-DETR training completed!")
            
            # Plot training curves
            plot_training_history(train_losses, val_losses)
            
            return train_losses, val_losses
            
        except Exception as e:
            print(f"❌ RF-DETR training failed: {e}")
            return None
    
    def evaluate_models(self):
        """Evaluate all trained models"""
        
        print("🧪 Evaluating trained models...")
        
        results = {}
        
        # Evaluate YOLO
        if self.yolo_detector:
            print("\\n📊 Evaluating YOLO model...")
            yolo_results = self.yolo_detector.evaluate_on_test_set()
            if yolo_results:
                results['yolo'] = {
                    'mAP50': yolo_results.box.map50,
                    'mAP50-95': yolo_results.box.map
                }
        
        # Evaluate RF-DETR (if test data available)
        if self.rfdetr_model:
            try:
                print("\\n📊 Evaluating RF-DETR model...")
                test_loader, _ = create_corrosion_dataloaders(
                    batch_size=4,
                    dataset_root=self.dataset_root
                )
                
                if len(test_loader) > 0:
                    evaluator = CorrosionEvaluator(self.rfdetr_model, test_loader)
                    eval_results = evaluator.evaluate_model()
                    evaluator.print_evaluation_report(eval_results)
                    results['rfdetr'] = eval_results
                
            except Exception as e:
                print(f"⚠️ RF-DETR evaluation skipped: {e}")
        
        return results
    
    def export_models(self):
        """Export trained models for deployment"""
        
        print("📦 Exporting models for deployment...")
        
        timestamp = int(time.time())
        export_dir = self.dataset_root.parent / 'models' / f'export_{timestamp}'
        export_dir.mkdir(parents=True, exist_ok=True)
        
        # Export YOLO model
        if self.yolo_detector and hasattr(self.yolo_detector.model, 'save'):
            yolo_path = export_dir / 'yolo_corrosion.pt'
            self.yolo_detector.model.save(str(yolo_path))
            print(f"✅ YOLO model exported: {yolo_path}")
        
        # Export RF-DETR model
        if self.rfdetr_model:
            export_model_for_production(self.rfdetr_model, str(export_dir / 'rfdetr_corrosion'))
            print(f"✅ RF-DETR model exported: {export_dir}")
        
        return export_dir
    
    def generate_training_report(self):
        """Generate comprehensive training report"""
        
        report_path = self.dataset_root.parent / f'training_report_{int(time.time())}.md'
        
        report_content = f"""# Corrosion Detection Training Report

## Dataset Information
- **Dataset Path**: `{self.dataset_root}`
- **Generated**: {pd.Timestamp.now()}

## Model Architecture
- **YOLOv11**: Real-time object detection
- **RF-DETR**: Fine-grained detection transformer
- **SAM2**: Precise segmentation (post-processing)

## Training Configuration
- **Device**: {"GPU" if torch.cuda.is_available() else "CPU"}
- **Framework**: PyTorch {torch.__version__}

## Class Labels
{chr(10).join(f"{i}. {name.replace('_', ' ').title()}" for i, name in enumerate(self.yolo_detector.class_names) if self.yolo_detector)}

## Usage Instructions

### For Inference:
```python
# Initialize detector
detector = YOLOCorrosionDetector("{self.dataset_root.parent / 'models' / 'best.pt'}")

# Run detection
detections, annotated = detector.predict("path/to/image.jpg")

# Print results
for det in detections:
    print(f"Found: {{det['class_name']}} (confidence: {{det['confidence']:.2f}})")
```

### For Dashboard:
```bash
streamlit run streamlit_corrosion_app.py
```

Generated by Corrosion Detection Training Workflow
"""
        
        with open(report_path, 'w') as f:
            f.write(report_content)
        
        print(f"📋 Training report generated: {report_path}")
        return report_path

# Initialize training workflow
print("🔧 Initializing Corrosion Training Workflow...")
workflow = CorrosionTrainingWorkflow()

# Setup training environment
dataset_stats = workflow.setup_training_environment()

print("\\n🎯 Training Workflow Ready!")
print("\\nQuick Start Commands:")
print("  📊 Check dataset: workflow.setup_training_environment()")
print("  🚀 Train YOLO: workflow.start_yolo_training(epochs=50, batch_size=8)")  
print("  🤖 Train RF-DETR: workflow.start_rfdetr_training(epochs=30, batch_size=4)")
print("  🧪 Evaluate: workflow.evaluate_models()")
print("  📦 Export: workflow.export_models()")
print("  📋 Report: workflow.generate_training_report()")

# Show next steps based on dataset status
train_available = dataset_stats['train']['images_count'] > 0
val_available = dataset_stats['validation']['images_count'] > 0

if train_available and val_available:
    print("\\n✅ Dataset is ready! You can start training immediately.")
    print("\\nRecommended workflow:")
    print("  1. workflow.start_yolo_training(epochs=50)  # Fast training")
    print("  2. workflow.evaluate_models()              # Check performance") 
    print("  3. workflow.export_models()                # Save for deployment")
else:
    print("\\n⚠️ Dataset setup required:")
    if not train_available:
        print("  - Add training images to: train/images/")
        print("  - Add training labels to: train/labels/")
    if not val_available:
        print("  - Add validation images to: validation/images/")
        print("  - Add validation labels to: validation/labels/")
    print("\\nDataset structure should be:")
    print("  Dataset/train/images/*.jpg")
    print("  Dataset/train/labels/*.txt (YOLO format)")
    print("  Dataset/validation/images/*.jpg") 
    print("  Dataset/validation/labels/*.txt")

In [ ]:
# SAM2 Integration for Precise Segmentation
class SAMCorrosionSegmenter:
    """SAM2-based corrosion segmentation for refined analysis"""
    
    def __init__(self, model_path=None):
        try:
            from segment_anything import sam_model_registry, SamPredictor
            
            # Initialize SAM model
            model_type = "vit_h"  # or "vit_l", "vit_b"
            if model_path and os.path.exists(model_path):
                sam = sam_model_registry[model_type](checkpoint=model_path)
            else:
                print("SAM model path not provided. Please download SAM weights.")
                sam = None
            
            if sam:
                sam.to(device=device)
                self.predictor = SamPredictor(sam)
            else:
                self.predictor = None
                
        except ImportError:
            print("Segment Anything not installed. Install with: pip install segment-anything")
            self.predictor = None
    
    def segment_corrosion(self, image, detections):
        """Segment corrosion areas using SAM2"""
        if self.predictor is None:
            return []
        
        segmentations = []
        self.predictor.set_image(image)
        
        for detection in detections:
            x1, y1, x2, y2 = detection['bbox']
            
            # Use bounding box as prompt for SAM
            input_box = np.array([x1, y1, x2, y2])
            
            try:
                masks, scores, logits = self.predictor.predict(
                    point_coords=None,
                    point_labels=None,
                    box=input_box[None, :],
                    multimask_output=False,
                )
                
                if len(masks) > 0:
                    mask = masks[0]
                    score = scores[0]
                    
                    # Calculate mask properties
                    mask_area = np.sum(mask)
                    bbox_area = (x2 - x1) * (y2 - y1)
                    fill_ratio = mask_area / bbox_area if bbox_area > 0 else 0
                    
                    segmentation = {
                        'mask': mask,
                        'score': score,
                        'area': mask_area,
                        'fill_ratio': fill_ratio,
                        'bbox': detection['bbox'],
                        'class': detection['class'],
                        'confidence': detection['confidence']
                    }
                    segmentations.append(segmentation)
                    
            except Exception as e:
                print(f"Segmentation failed for detection: {e}")
                continue
        
        return segmentations
    
    def refine_detections(self, image, detections):
        """Refine bounding boxes using segmentation masks"""
        segmentations = self.segment_corrosion(image, detections)
        refined_detections = []
        
        for seg in segmentations:
            mask = seg['mask']
            
            # Find tight bounding box around mask
            y_indices, x_indices = np.where(mask)
            if len(x_indices) > 0 and len(y_indices) > 0:
                x_min, x_max = x_indices.min(), x_indices.max()
                y_min, y_max = y_indices.min(), y_indices.max()
                
                refined_detection = {
                    'bbox': [x_min, y_min, x_max, y_max],
                    'refined_bbox': [x_min, y_min, x_max, y_max],
                    'original_bbox': seg['bbox'],
                    'mask': mask,
                    'confidence': seg['confidence'],
                    'class': seg['class'],
                    'area': seg['area'],
                    'fill_ratio': seg['fill_ratio'],
                    'segmentation_score': seg['score']
                }
                refined_detections.append(refined_detection)
        
        return refined_detections

In [ ]:
# Real-time Processing Pipeline
class RealTimeCorrosionPipeline:
    """Complete real-time corrosion detection and analysis pipeline"""
    
    def __init__(self, yolo_model_path=None, sam_model_path=None, rfdetr_model_path=None):
        # Initialize detection models
        self.yolo_detector = YOLOCorrosionDetector(yolo_model_path)
        self.sam_segmenter = SAMCorrosionSegmenter(sam_model_path)
        
        # Initialize RF-DETR if available
        if rfdetr_model_path and os.path.exists(rfdetr_model_path):
            self.rfdetr = RFDETR()
            self.rfdetr.load_state_dict(torch.load(rfdetr_model_path, map_location=device))
            self.rfdetr.to(device)
            self.rfdetr.eval()
        else:
            self.rfdetr = None
        
        # Performance tracking
        self.fps_counter = 0
        self.start_time = time.time()
        self.processing_times = []
        
        # Corrosion tracking
        self.corrosion_history = []
        self.severity_tracker = {}
        
    def process_frame(self, frame):
        """Process single frame through the complete pipeline"""
        start_time = time.time()
        
        # Step 1: YOLO Detection
        detections, annotated_frame = self.yolo_detector.predict(frame, return_annotated=True)
        
        # Step 2: SAM Segmentation (if detections found)
        refined_detections = []
        if detections and self.sam_segmenter.predictor:
            refined_detections = self.sam_segmenter.refine_detections(frame, detections)
        
        # Step 3: RF-DETR Analysis (if model available)
        rfdetr_results = None
        if self.rfdetr and detections:
            rfdetr_results = self.analyze_with_rfdetr(frame)
        
        # Step 4: Comprehensive Analysis
        analysis_results = self.analyze_corrosion_patterns(
            detections, refined_detections, rfdetr_results
        )
        
        # Update tracking
        processing_time = time.time() - start_time
        self.processing_times.append(processing_time)
        self.fps_counter += 1
        
        # Calculate FPS
        current_time = time.time()
        elapsed = current_time - self.start_time
        fps = self.fps_counter / elapsed if elapsed > 0 else 0
        
        return {
            'annotated_frame': annotated_frame,
            'detections': detections,
            'refined_detections': refined_detections,
            'analysis': analysis_results,
            'performance': {
                'fps': fps,
                'processing_time': processing_time,
                'avg_processing_time': np.mean(self.processing_times[-30:]) if self.processing_times else 0
            }
        }
    
    def analyze_with_rfdetr(self, frame):
        """Analyze frame using RF-DETR model"""
        if self.rfdetr is None:
            return None
        
        # Preprocess frame
        transform = get_val_transforms()
        input_tensor = transform(image=frame)['image'].unsqueeze(0).to(device)
        
        with torch.no_grad():
            outputs = self.rfdetr(input_tensor)
        
        return outputs
    
    def analyze_corrosion_patterns(self, detections, refined_detections, rfdetr_results):
        """Comprehensive corrosion pattern analysis"""
        analysis = {
            'total_corrosion_areas': len(detections),
            'corrosion_types': {},
            'severity_distribution': {},
            'coverage_percentage': 0,
            'risk_assessment': 'low',
            'recommendations': []
        }
        
        if not detections:
            return analysis
        
        # Analyze corrosion types
        for detection in detections:
            class_name = detection['class_name']
            analysis['corrosion_types'][class_name] = analysis['corrosion_types'].get(class_name, 0) + 1
        
        # Calculate coverage if refined detections available
        if refined_detections:
            total_area = 0
            for refined in refined_detections:
                total_area += refined['area']
            
            # Assume frame dimensions for coverage calculation
            frame_area = 640 * 640  # Default YOLO input size
            analysis['coverage_percentage'] = (total_area / frame_area) * 100
        
        # Risk assessment
        if analysis['coverage_percentage'] > 20:
            analysis['risk_assessment'] = 'critical'
            analysis['recommendations'].append('Immediate intervention required')
        elif analysis['coverage_percentage'] > 10:
            analysis['risk_assessment'] = 'high'
            analysis['recommendations'].append('Schedule maintenance within 24 hours')
        elif analysis['coverage_percentage'] > 5:
            analysis['risk_assessment'] = 'medium'
            analysis['recommendations'].append('Monitor closely, schedule inspection')
        else:
            analysis['risk_assessment'] = 'low'
            analysis['recommendations'].append('Continue regular monitoring')
        
        # Check for critical corrosion types
        critical_types = ['stress_corrosion', 'crevice_corrosion']
        for ctype in critical_types:
            if ctype in analysis['corrosion_types']:
                if analysis['risk_assessment'] not in ['critical', 'high']:
                    analysis['risk_assessment'] = 'high'
                analysis['recommendations'].append(f'Critical {ctype.replace("_", " ")} detected')
        
        return analysis

In [ ]:
# Four-Panel Dashboard Implementation
class CorrosionDashboard:
    """Real-time dashboard for corrosion monitoring system"""
    
    def __init__(self):
        self.performance_data = []
        self.corrosion_history = []
        self.alerts = []
        self.system_stats = {
            'total_frames_processed': 0,
            'total_detections': 0,
            'uptime': 0,
            'last_critical_alert': None
        }
        
    def update_performance_data(self, fps, processing_time, detections_count):
        """Update performance metrics"""
        timestamp = time.time()
        self.performance_data.append({
            'timestamp': timestamp,
            'fps': fps,
            'processing_time': processing_time,
            'detections_count': detections_count
        })
        
        # Keep only last 100 data points
        if len(self.performance_data) > 100:
            self.performance_data.pop(0)
    
    def update_corrosion_analysis(self, analysis_results):
        """Update corrosion analysis data"""
        timestamp = time.time()
        self.corrosion_history.append({
            'timestamp': timestamp,
            'total_areas': analysis_results['total_corrosion_areas'],
            'coverage_percentage': analysis_results['coverage_percentage'],
            'risk_level': analysis_results['risk_assessment'],
            'corrosion_types': analysis_results['corrosion_types'].copy()
        })
        
        # Generate alerts for critical conditions
        if analysis_results['risk_assessment'] in ['critical', 'high']:
            alert = {
                'timestamp': timestamp,
                'level': analysis_results['risk_assessment'],
                'message': f"High corrosion activity detected: {analysis_results['coverage_percentage']:.1f}% coverage",
                'recommendations': analysis_results['recommendations']
            }
            self.alerts.append(alert)
            self.system_stats['last_critical_alert'] = timestamp
        
        # Keep only last 50 history points
        if len(self.corrosion_history) > 50:
            self.corrosion_history.pop(0)
        
        # Keep only last 20 alerts
        if len(self.alerts) > 20:
            self.alerts.pop(0)
    
    def create_dashboard_plots(self):
        """Create all four dashboard panels"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'System Performance Metrics',
                'Corrosion Detection Trends', 
                'Material Health Score & Alerts',
                'Corrosion Type Distribution'
            ),
            specs=[
                [{"secondary_y": True}, {"secondary_y": True}],
                [{"secondary_y": False}, {"type": "pie"}]
            ]
        )
        
        # Panel 1: System Performance
        if self.performance_data:
            timestamps = [pd.to_datetime(p['timestamp'], unit='s') for p in self.performance_data]
            fps_values = [p['fps'] for p in self.performance_data]
            processing_times = [p['processing_time'] * 1000 for p in self.performance_data]  # Convert to ms
            
            fig.add_trace(
                go.Scatter(x=timestamps, y=fps_values, name="FPS", line=dict(color='blue')),
                row=1, col=1
            )
            fig.add_trace(
                go.Scatter(x=timestamps, y=processing_times, name="Processing Time (ms)", 
                          line=dict(color='red'), yaxis="y2"),
                row=1, col=1, secondary_y=True
            )
        
        # Panel 2: Corrosion Trends
        if self.corrosion_history:
            timestamps = [pd.to_datetime(h['timestamp'], unit='s') for h in self.corrosion_history]
            coverage_values = [h['coverage_percentage'] for h in self.corrosion_history]
            total_areas = [h['total_areas'] for h in self.corrosion_history]
            
            fig.add_trace(
                go.Scatter(x=timestamps, y=coverage_values, name="Coverage %", 
                          line=dict(color='orange'), fill='tonexty'),
                row=1, col=2
            )
            fig.add_trace(
                go.Scatter(x=timestamps, y=total_areas, name="Detection Count", 
                          line=dict(color='green'), yaxis="y4"),
                row=1, col=2, secondary_y=True
            )
        
        # Panel 3: Health Score and Alerts
        if self.corrosion_history:
            # Calculate health score (inverse of risk)
            risk_mapping = {'low': 85, 'medium': 65, 'high': 35, 'critical': 10}
            health_scores = [risk_mapping.get(h['risk_level'], 50) for h in self.corrosion_history]
            timestamps = [pd.to_datetime(h['timestamp'], unit='s') for h in self.corrosion_history]
            
            # Color based on health score
            colors = ['green' if score > 70 else 'yellow' if score > 40 else 'red' for score in health_scores]
            
            fig.add_trace(
                go.Scatter(x=timestamps, y=health_scores, mode='lines+markers',
                          name="Material Health Score", 
                          line=dict(color='purple', width=3),
                          marker=dict(size=8)),
                row=2, col=1
            )
        
        # Panel 4: Corrosion Type Distribution
        if self.corrosion_history and self.corrosion_history[-1]['corrosion_types']:
            latest_types = self.corrosion_history[-1]['corrosion_types']
            labels = list(latest_types.keys())
            values = list(latest_types.values())
            
            fig.add_trace(
                go.Pie(labels=labels, values=values, name="Corrosion Types",
                      hole=0.3, textinfo='label+percent'),
                row=2, col=2
            )
        
        # Update layout
        fig.update_layout(
            title="Real-time Corrosion Monitoring Dashboard",
            height=800,
            showlegend=True,
            template="plotly_white"
        )
        
        # Update axes labels
        fig.update_xaxes(title_text="Time", row=1, col=1)
        fig.update_yaxes(title_text="FPS", row=1, col=1)
        fig.update_yaxes(title_text="Processing Time (ms)", row=1, col=1, secondary_y=True)
        
        fig.update_xaxes(title_text="Time", row=1, col=2)
        fig.update_yaxes(title_text="Coverage %", row=1, col=2)
        fig.update_yaxes(title_text="Detection Count", row=1, col=2, secondary_y=True)
        
        fig.update_xaxes(title_text="Time", row=2, col=1)
        fig.update_yaxes(title_text="Health Score", row=2, col=1)
        
        return fig
    
    def get_current_stats(self):
        """Get current system statistics"""
        current_stats = self.system_stats.copy()
        
        if self.performance_data:
            current_stats['current_fps'] = self.performance_data[-1]['fps']
            current_stats['avg_processing_time'] = np.mean([p['processing_time'] for p in self.performance_data[-10:]])
        
        if self.corrosion_history:
            current_stats['current_coverage'] = self.corrosion_history[-1]['coverage_percentage']
            current_stats['current_risk'] = self.corrosion_history[-1]['risk_level']
            current_stats['health_score'] = {'low': 85, 'medium': 65, 'high': 35, 'critical': 10}.get(
                self.corrosion_history[-1]['risk_level'], 50)
        
        current_stats['active_alerts'] = len([a for a in self.alerts if time.time() - a['timestamp'] < 3600])  # Last hour
        
        return current_stats

In [ ]:
# Streamlit Dashboard Application
def create_streamlit_dashboard():
    """Create Streamlit web dashboard for real-time monitoring"""
    
    st.set_page_config(
        page_title="Corrosion Detection Dashboard",
        page_icon="🔬",
        layout="wide",
        initial_sidebar_state="expanded"
    )
    
    st.title("🔬 Real-time Corrosion Detection System")
    st.markdown("### Advanced Computer Vision Pipeline for R&D Material Science Monitoring")
    
    # Initialize session state
    if 'pipeline' not in st.session_state:
        st.session_state.pipeline = RealTimeCorrosionPipeline()
        st.session_state.dashboard = CorrosionDashboard()
        st.session_state.is_running = False
        st.session_state.video_source = None
    
    # Sidebar configuration
    st.sidebar.header("⚙️ System Configuration")
    
    # Video source selection
    source_type = st.sidebar.selectbox(
        "Select Video Source",
        ["Webcam", "Video File", "Image File", "Demo Mode"]
    )
    
    if source_type == "Video File":
        uploaded_video = st.sidebar.file_uploader("Upload Video", type=['mp4', 'avi', 'mov'])
        if uploaded_video:
            st.session_state.video_source = uploaded_video
    elif source_type == "Image File":
        uploaded_image = st.sidebar.file_uploader("Upload Image", type=['jpg', 'jpeg', 'png'])
        if uploaded_image:
            st.session_state.video_source = uploaded_image
    elif source_type == "Webcam":
        st.session_state.video_source = 0
    else:  # Demo Mode
        st.session_state.video_source = "demo"
    
    # Model configuration
    st.sidebar.subheader("🤖 Model Settings")
    confidence_threshold = st.sidebar.slider("Confidence Threshold", 0.1, 1.0, 0.5, 0.1)
    iou_threshold = st.sidebar.slider("IoU Threshold", 0.1, 1.0, 0.4, 0.1)
    
    # Update thresholds
    st.session_state.pipeline.yolo_detector.conf_threshold = confidence_threshold
    st.session_state.pipeline.yolo_detector.iou_threshold = iou_threshold
    
    # Control buttons
    col1, col2, col3 = st.sidebar.columns(3)
    with col1:
        if st.button("▶️ Start"):
            st.session_state.is_running = True
    with col2:
        if st.button("⏸️ Pause"):
            st.session_state.is_running = False
    with col3:
        if st.button("🔄 Reset"):
            st.session_state.dashboard = CorrosionDashboard()
    
    # Main dashboard layout
    if st.session_state.is_running and st.session_state.video_source is not None:
        
        # Create main content area
        video_col, stats_col = st.columns([2, 1])
        
        with video_col:
            st.subheader("📹 Live Video Feed")
            video_placeholder = st.empty()
            
            # Process video/image
            if source_type == "Image File" and st.session_state.video_source:
                # Single image processing
                image = np.array(Image.open(st.session_state.video_source))
                results = st.session_state.pipeline.process_frame(image)
                
                # Update dashboard
                st.session_state.dashboard.update_performance_data(
                    results['performance']['fps'],
                    results['performance']['processing_time'],
                    len(results['detections'])
                )
                st.session_state.dashboard.update_corrosion_analysis(results['analysis'])
                
                # Display annotated image
                video_placeholder.image(results['annotated_frame'], channels="RGB", use_column_width=True)
            
            elif source_type == "Demo Mode":
                # Demo with synthetic data
                demo_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
                results = {
                    'annotated_frame': demo_image,
                    'detections': [
                        {'class_name': 'pitting_corrosion', 'confidence': 0.85, 'bbox': [100, 100, 200, 200]},
                        {'class_name': 'uniform_corrosion', 'confidence': 0.72, 'bbox': [300, 150, 400, 250]}
                    ],
                    'analysis': {
                        'total_corrosion_areas': 2,
                        'coverage_percentage': 12.5,
                        'risk_assessment': 'high',
                        'corrosion_types': {'pitting_corrosion': 1, 'uniform_corrosion': 1},
                        'recommendations': ['Schedule maintenance within 24 hours']
                    },
                    'performance': {'fps': 24.5, 'processing_time': 0.041}
                }
                
                st.session_state.dashboard.update_performance_data(24.5, 0.041, 2)
                st.session_state.dashboard.update_corrosion_analysis(results['analysis'])
                video_placeholder.image(demo_image, channels="RGB", use_column_width=True)
        
        with stats_col:
            st.subheader("📊 Real-time Stats")
            
            # Current statistics
            current_stats = st.session_state.dashboard.get_current_stats()
            
            # Metrics display
            col1, col2 = st.columns(2)
            with col1:
                st.metric("FPS", f"{current_stats.get('current_fps', 0):.1f}")
                st.metric("Health Score", f"{current_stats.get('health_score', 50)}")
            with col2:
                st.metric("Coverage %", f"{current_stats.get('current_coverage', 0):.1f}%")
                st.metric("Active Alerts", current_stats.get('active_alerts', 0))
            
            # Risk assessment
            risk_level = current_stats.get('current_risk', 'unknown')
            risk_color = {
                'low': 'green', 'medium': 'orange', 
                'high': 'red', 'critical': 'darkred'
            }.get(risk_level, 'gray')
            
            st.markdown(f"**Risk Level:** <span style='color: {risk_color}'>{risk_level.upper()}</span>", 
                       unsafe_allow_html=True)
        
        # Four-panel dashboard
        st.subheader("📈 Analysis Dashboard")
        
        # Generate and display plots
        dashboard_fig = st.session_state.dashboard.create_dashboard_plots()
        st.plotly_chart(dashboard_fig, use_container_width=True)
        
        # Recent alerts
        if st.session_state.dashboard.alerts:
            st.subheader("🚨 Recent Alerts")
            for alert in st.session_state.dashboard.alerts[-5:]:  # Show last 5 alerts
                alert_time = pd.to_datetime(alert['timestamp'], unit='s').strftime('%H:%M:%S')
                alert_color = 'red' if alert['level'] == 'critical' else 'orange'
                st.markdown(
                    f"**{alert_time}** - <span style='color: {alert_color}'>{alert['level'].upper()}</span>: {alert['message']}",
                    unsafe_allow_html=True
                )
                for rec in alert['recommendations']:
                    st.markdown(f"  • {rec}")
    
    else:
        # Welcome screen
        st.info("👆 Configure your video source in the sidebar and click 'Start' to begin monitoring")
        
        # Show system capabilities
        col1, col2, col3 = st.columns(3)
        with col1:
            st.markdown("""
            ### 🔧 Core Pipeline
            - RF-DETR for fine-grained detection
            - YOLOv11 for real-time processing  
            - SAM2 for precise segmentation
            """)
        with col2:
            st.markdown("""
            ### 📊 Dashboard Features
            - Real-time performance metrics
            - Corrosion trend analysis
            - Health score monitoring
            - Alert management system
            """)
        with col3:
            st.markdown("""
            ### 🧪 R&D Benefits
            - Early corrosion detection
            - 24/7 automated monitoring
            - Data-driven insights
            - Predictive maintenance
            """)

In [ ]:
# Training Pipeline for RF-DETR
class RFDETRTrainer:
    """Training pipeline for RF-DETR model"""
    
    def __init__(self, model, train_loader, val_loader, num_epochs=100, learning_rate=1e-4):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.num_epochs = num_epochs
        
        # Optimizer and scheduler
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(), 
            lr=learning_rate, 
            weight_decay=1e-4
        )
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=num_epochs
        )
        
        # Loss function
        self.criterion = RFDETRLoss(num_classes=6)
        
        # Training history
        self.train_losses = []
        self.val_losses = []
        self.best_val_loss = float('inf')
        
    def hungarian_matching(self, pred_boxes, pred_classes, target_boxes, target_classes):
        """Hungarian matching for object detection"""
        from scipy.optimize import linear_sum_assignment
        
        batch_size = pred_boxes.shape[0]
        indices = []
        
        for b in range(batch_size):
            # Compute cost matrix
            pred_b = pred_boxes[b]  # [num_queries, 4]
            target_b = target_boxes[b]  # [num_targets, 4]
            pred_cls_b = pred_classes[b]  # [num_queries, num_classes]
            target_cls_b = target_classes[b]  # [num_targets]
            
            if len(target_b) == 0:
                indices.append((torch.tensor([]), torch.tensor([])))
                continue
            
            # Classification cost
            pred_probs = F.softmax(pred_cls_b, dim=-1)
            cls_cost = -pred_probs[:, target_cls_b]
            
            # L1 cost
            l1_cost = torch.cdist(pred_b, target_b, p=1)
            
            # IoU cost (approximate with GIoU)
            giou_cost = -self.criterion.generalized_box_iou(pred_b, target_b)
            
            # Combined cost
            cost = (self.criterion.cost_class * cls_cost + 
                   self.criterion.cost_bbox * l1_cost + 
                   self.criterion.cost_giou * giou_cost)
            
            # Hungarian algorithm
            pred_idx, target_idx = linear_sum_assignment(cost.cpu().numpy())
            indices.append((torch.tensor(pred_idx), torch.tensor(target_idx)))
        
        return indices
    
    def compute_loss(self, outputs, targets):
        """Compute RF-DETR loss with Hungarian matching"""
        pred_boxes = outputs['bbox_coords']  # [B, num_queries, 4]
        pred_classes = outputs['class_logits']  # [B, num_queries, num_classes+1]
        pred_severity = outputs['severity_logits']  # [B, num_queries, 5]
        
        batch_size = pred_boxes.shape[0]
        total_loss = 0
        
        for b in range(batch_size):
            target = targets[b]
            target_boxes = target['boxes']
            target_classes = target['labels']
            target_severity = target.get('severity', torch.ones_like(target_classes))
            
            if len(target_boxes) == 0:
                # No targets, only classification loss for background
                bg_loss = F.cross_entropy(
                    pred_classes[b], 
                    torch.zeros(pred_classes[b].shape[0], dtype=torch.long, device=device)
                )
                total_loss += bg_loss
                continue
            
            # Hungarian matching
            indices = self.hungarian_matching(
                pred_boxes[b:b+1], pred_classes[b:b+1], 
                target_boxes.unsqueeze(0), target_classes.unsqueeze(0)
            )
            pred_idx, target_idx = indices[0]
            
            if len(pred_idx) == 0:
                continue
            
            # Selected predictions and targets
            selected_pred_boxes = pred_boxes[b][pred_idx]
            selected_pred_classes = pred_classes[b][pred_idx]
            selected_pred_severity = pred_severity[b][pred_idx]
            selected_target_boxes = target_boxes[target_idx]
            selected_target_classes = target_classes[target_idx]
            selected_target_severity = target_severity[target_idx]
            
            # Classification loss
            cls_loss = F.cross_entropy(selected_pred_classes, selected_target_classes)
            
            # Bounding box loss
            bbox_loss = F.l1_loss(selected_pred_boxes, selected_target_boxes)
            
            # GIoU loss
            giou_loss = 1 - self.criterion.generalized_box_iou(
                selected_pred_boxes, selected_target_boxes
            ).diag().mean()
            
            # Severity loss
            severity_loss = F.cross_entropy(selected_pred_severity, selected_target_severity)
            
            # Background classification loss for unmatched predictions
            unmatched_mask = torch.ones(pred_classes[b].shape[0], dtype=torch.bool, device=device)
            unmatched_mask[pred_idx] = False
            if unmatched_mask.any():
                bg_loss = F.cross_entropy(
                    pred_classes[b][unmatched_mask],
                    torch.zeros(unmatched_mask.sum(), dtype=torch.long, device=device)
                )
            else:
                bg_loss = 0
            
            # Combine losses
            sample_loss = (cls_loss + 
                          self.criterion.cost_bbox * bbox_loss + 
                          self.criterion.cost_giou * giou_loss + 
                          self.criterion.cost_severity * severity_loss + 
                          0.1 * bg_loss)
            
            total_loss += sample_loss
        
        return total_loss / batch_size
    
    def train_epoch(self):
        """Train for one epoch"""
        self.model.train()
        epoch_loss = 0
        num_batches = 0
        
        for batch_idx, (images, targets) in enumerate(self.train_loader):
            images = images.to(device)
            targets = [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]
            
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.compute_loss(outputs, targets)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.1)
            self.optimizer.step()
            
            epoch_loss += loss.item()
            num_batches += 1
            
            if batch_idx % 10 == 0:
                print(f"Batch {batch_idx}/{len(self.train_loader)}, Loss: {loss.item():.4f}")
        
        return epoch_loss / num_batches
    
    def validate_epoch(self):
        """Validate for one epoch"""
        self.model.eval()
        epoch_loss = 0
        num_batches = 0
        
        with torch.no_grad():
            for images, targets in self.val_loader:
                images = images.to(device)
                targets = [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]
                
                outputs = self.model(images)
                loss = self.compute_loss(outputs, targets)
                
                epoch_loss += loss.item()
                num_batches += 1
        
        return epoch_loss / num_batches
    
    def train(self):
        """Complete training loop"""
        print("Starting RF-DETR training...")
        
        for epoch in range(self.num_epochs):
            print(f"\nEpoch {epoch+1}/{self.num_epochs}")
            
            # Training
            train_loss = self.train_epoch()
            self.train_losses.append(train_loss)
            
            # Validation
            val_loss = self.validate_epoch()
            self.val_losses.append(val_loss)
            
            # Learning rate scheduling
            self.scheduler.step()
            
            print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
            
            # Save best model
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                torch.save(self.model.state_dict(), 'best_rfdetr_model.pth')
                print("💾 New best model saved!")
        
        print("Training completed!")
        return self.train_losses, self.val_losses

In [ ]:
# Model Evaluation and Metrics
class CorrosionEvaluator:
    """Evaluation metrics for corrosion detection models"""
    
    def __init__(self, model, dataloader, iou_threshold=0.5):
        self.model = model
        self.dataloader = dataloader
        self.iou_threshold = iou_threshold
        self.class_names = [
            'background', 'uniform_corrosion', 'pitting_corrosion', 
            'crevice_corrosion', 'stress_corrosion', 'galvanic_corrosion', 'erosion_corrosion'
        ]
    
    def calculate_ap(self, precisions, recalls):
        """Calculate Average Precision using 11-point interpolation"""
        ap = 0
        for t in np.arange(0, 1.1, 0.1):
            if np.sum(recalls >= t) == 0:
                p = 0
            else:
                p = np.max(precisions[recalls >= t])
            ap += p / 11
        return ap
    
    def evaluate_detection(self, pred_boxes, pred_scores, pred_classes, target_boxes, target_classes):
        """Evaluate detection performance for single image"""
        if len(pred_boxes) == 0:
            return {'tp': 0, 'fp': 0, 'fn': len(target_boxes)}
        
        if len(target_boxes) == 0:
            return {'tp': 0, 'fp': len(pred_boxes), 'fn': 0}
        
        # Calculate IoU matrix
        iou_matrix = self.box_iou(pred_boxes, target_boxes)
        
        # Find best matches
        tp = 0
        matched_targets = set()
        matched_preds = set()
        
        # Sort predictions by score (descending)
        sorted_indices = np.argsort(pred_scores)[::-1]
        
        for pred_idx in sorted_indices:
            best_iou = 0
            best_target_idx = -1
            
            for target_idx in range(len(target_boxes)):
                if target_idx in matched_targets:
                    continue
                
                # Check class match and IoU
                if (pred_classes[pred_idx] == target_classes[target_idx] and 
                    iou_matrix[pred_idx, target_idx] >= self.iou_threshold and
                    iou_matrix[pred_idx, target_idx] > best_iou):
                    best_iou = iou_matrix[pred_idx, target_idx]
                    best_target_idx = target_idx
            
            if best_target_idx != -1:
                tp += 1
                matched_targets.add(best_target_idx)
                matched_preds.add(pred_idx)
        
        fp = len(pred_boxes) - tp
        fn = len(target_boxes) - tp
        
        return {'tp': tp, 'fp': fp, 'fn': fn}
    
    def box_iou(self, boxes1, boxes2):
        """Calculate IoU between two sets of boxes"""
        def box_area(boxes):
            return (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        
        area1 = box_area(boxes1)
        area2 = box_area(boxes2)
        
        lt = np.maximum(boxes1[:, None, :2], boxes2[:, :2])
        rb = np.minimum(boxes1[:, None, 2:], boxes2[:, 2:])
        
        wh = np.clip(rb - lt, 0, None)
        inter = wh[:, :, 0] * wh[:, :, 1]
        
        iou = inter / (area1[:, None] + area2 - inter)
        return iou
    
    def evaluate_model(self):
        """Comprehensive model evaluation"""
        self.model.eval()
        
        all_detections = []
        all_targets = []
        class_metrics = {cls: {'tp': 0, 'fp': 0, 'fn': 0} for cls in range(len(self.class_names))}
        
        print("Evaluating model...")
        with torch.no_grad():
            for batch_idx, (images, targets) in enumerate(self.dataloader):
                images = images.to(device)
                
                outputs = self.model(images)
                
                # Process each image in batch
                for i in range(len(images)):
                    # Extract predictions
                    pred_boxes = outputs['bbox_coords'][i].cpu().numpy()
                    pred_classes_logits = outputs['class_logits'][i].cpu().numpy()
                    pred_classes = np.argmax(pred_classes_logits, axis=1)
                    pred_scores = np.max(F.softmax(torch.tensor(pred_classes_logits), dim=1).numpy(), axis=1)
                    
                    # Filter out background predictions and low confidence
                    valid_mask = (pred_classes != 0) & (pred_scores > 0.1)
                    pred_boxes = pred_boxes[valid_mask]
                    pred_classes = pred_classes[valid_mask]
                    pred_scores = pred_scores[valid_mask]
                    
                    # Convert normalized boxes to pixel coordinates (assuming 512x512)
                    pred_boxes = pred_boxes * 512
                    pred_boxes[:, [0, 2]] = np.clip(pred_boxes[:, [0, 2]], 0, 512)
                    pred_boxes[:, [1, 3]] = np.clip(pred_boxes[:, [1, 3]], 0, 512)
                    
                    # Extract targets
                    target = targets[i]
                    target_boxes = target['boxes'].cpu().numpy() * 512
                    target_classes = target['labels'].cpu().numpy()
                    
                    # Evaluate detection for this image
                    result = self.evaluate_detection(pred_boxes, pred_scores, pred_classes, 
                                                   target_boxes, target_classes)
                    
                    # Update class-wise metrics
                    for cls in np.unique(np.concatenate([pred_classes, target_classes])):
                        if cls in class_metrics:
                            class_metrics[cls]['tp'] += result['tp']
                            class_metrics[cls]['fp'] += result['fp']
                            class_metrics[cls]['fn'] += result['fn']
                
                if batch_idx % 10 == 0:
                    print(f"Processed {batch_idx}/{len(self.dataloader)} batches")
        
        # Calculate metrics
        results = self.calculate_metrics(class_metrics)
        return results
    
    def calculate_metrics(self, class_metrics):
        """Calculate precision, recall, F1, and mAP"""
        results = {}
        total_tp = total_fp = total_fn = 0
        
        for cls_id, cls_name in enumerate(self.class_names):
            if cls_id == 0:  # Skip background
                continue
            
            metrics = class_metrics[cls_id]
            tp, fp, fn = metrics['tp'], metrics['fp'], metrics['fn']
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            results[cls_name] = {
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'tp': tp,
                'fp': fp,
                'fn': fn
            }
            
            total_tp += tp
            total_fp += fp
            total_fn += fn
        
        # Overall metrics
        overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
        overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
        overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0
        
        results['overall'] = {
            'precision': overall_precision,
            'recall': overall_recall,
            'f1': overall_f1,
            'tp': total_tp,
            'fp': total_fp,
            'fn': total_fn
        }
        
        # mAP calculation (simplified)
        map_score = np.mean([results[cls]['f1'] for cls in results if cls != 'overall'])
        results['mAP'] = map_score
        
        return results
    
    def print_evaluation_report(self, results):
        """Print detailed evaluation report"""
        print("\n" + "="*80)
        print("CORROSION DETECTION MODEL EVALUATION REPORT")
        print("="*80)
        
        print(f"\nOverall Performance:")
        print(f"  Precision: {results['overall']['precision']:.3f}")
        print(f"  Recall:    {results['overall']['recall']:.3f}")
        print(f"  F1-Score:  {results['overall']['f1']:.3f}")
        print(f"  mAP:       {results['mAP']:.3f}")
        
        print(f"\nPer-Class Performance:")
        print("-" * 80)
        print(f"{'Class':<20} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'TP':<5} {'FP':<5} {'FN':<5}")
        print("-" * 80)
        
        for cls_name, metrics in results.items():
            if cls_name in ['overall', 'mAP']:
                continue
            print(f"{cls_name:<20} {metrics['precision']:<10.3f} {metrics['recall']:<10.3f} "
                  f"{metrics['f1']:<10.3f} {metrics['tp']:<5d} {metrics['fp']:<5d} {metrics['fn']:<5d}")
        
        print("="*80)

In [ ]:
# Demo and Visualization Functions
def create_demo_data():
    """Create synthetic demo data for testing the pipeline"""
    
    # Generate synthetic corrosion images
    demo_images = []
    demo_annotations = []
    
    for i in range(10):
        # Create base image with texture
        image = np.random.randint(100, 150, (512, 512, 3), dtype=np.uint8)
        
        # Add corrosion patterns
        annotations = {
            'boxes': [],
            'labels': [],
            'severity': []
        }
        
        # Add random corrosion spots
        num_spots = np.random.randint(1, 5)
        for _ in range(num_spots):
            # Random position and size
            x = np.random.randint(50, 450)
            y = np.random.randint(50, 450)
            w = np.random.randint(30, 100)
            h = np.random.randint(30, 100)
            
            # Ensure within bounds
            x2 = min(x + w, 512)
            y2 = min(y + h, 512)
            
            # Add corrosion pattern to image
            corrosion_type = np.random.randint(1, 7)  # 1-6 corrosion types
            
            if corrosion_type == 1:  # Uniform corrosion - reddish brown
                image[y:y2, x:x2] = [139, 69, 19]  # Brown
            elif corrosion_type == 2:  # Pitting - dark spots
                cv2.circle(image, (x + w//2, y + h//2), min(w, h)//2, (50, 50, 50), -1)
            elif corrosion_type == 3:  # Crevice - linear patterns
                for j in range(y, y2, 5):
                    cv2.line(image, (x, j), (x2, j), (80, 80, 80), 2)
            elif corrosion_type == 4:  # Stress corrosion - crack-like
                cv2.line(image, (x, y), (x2, y2), (60, 60, 60), 3)
                cv2.line(image, (x, y2), (x2, y), (60, 60, 60), 3)
            elif corrosion_type == 5:  # Galvanic - metallic appearance
                image[y:y2, x:x2] = [192, 192, 192]  # Silver
            else:  # Erosion - rough texture
                noise = np.random.randint(-30, 30, (y2-y, x2-x, 3))
                image[y:y2, x:x2] = np.clip(image[y:y2, x:x2] + noise, 0, 255)
            
            # Add to annotations (normalize coordinates)
            annotations['boxes'].append([x/512, y/512, x2/512, y2/512])
            annotations['labels'].append(corrosion_type)
            annotations['severity'].append(np.random.randint(1, 5))  # 1-4 severity
        
        demo_images.append(image)
        demo_annotations.append(annotations)
    
    return demo_images, demo_annotations

def visualize_detections(image, detections, title="Corrosion Detection Results"):
    """Visualize detection results with bounding boxes and labels"""
    
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.title(title, fontsize=16, fontweight='bold')
    plt.axis('off')
    
    # Color mapping for different corrosion types
    colors = {
        'uniform_corrosion': 'red',
        'pitting_corrosion': 'blue', 
        'crevice_corrosion': 'green',
        'stress_corrosion': 'yellow',
        'galvanic_corrosion': 'magenta',
        'erosion_corrosion': 'cyan'
    }
    
    for detection in detections:
        x1, y1, x2, y2 = detection['bbox']
        class_name = detection['class_name']
        confidence = detection['confidence']
        
        # Draw bounding box
        color = colors.get(class_name, 'white')
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                           fill=False, color=color, linewidth=2)
        plt.gca().add_patch(rect)
        
        # Add label
        label = f"{class_name.replace('_', ' ').title()}\n{confidence:.2f}"
        plt.text(x1, y1-10, label, color=color, fontsize=10, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor='black', alpha=0.7))
    
    plt.tight_layout()
    plt.show()

def plot_training_history(train_losses, val_losses):
    """Plot training and validation loss curves"""
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(val_losses, label='Validation Loss', color='red')
    plt.title('Model Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(np.array(train_losses) - np.array(val_losses), 
             label='Train-Val Gap', color='green')
    plt.title('Overfitting Analysis')
    plt.xlabel('Epoch')
    plt.ylabel('Loss Difference')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

def create_confusion_matrix(true_labels, pred_labels, class_names):
    """Create and plot confusion matrix"""
    
    cm = confusion_matrix(true_labels, pred_labels)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Corrosion Detection Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    return cm

In [ ]:
# Initialize and Test the Complete System
def initialize_system():
    """Initialize the complete corrosion detection system"""
    
    print("🔧 Initializing Corrosion Detection System...")
    print("-" * 50)
    
    # Initialize models
    print("📥 Loading RF-DETR model...")
    rfdetr_model = RFDETR(num_classes=6, num_queries=100)
    print(f"✅ RF-DETR initialized with {sum(p.numel() for p in rfdetr_model.parameters()):,} parameters")
    
    # Initialize YOLO detector
    print("📥 Loading YOLO detector...")
    yolo_detector = YOLOCorrosionDetector()
    print("✅ YOLOv11 detector initialized")
    
    # Initialize SAM segmenter
    print("📥 Loading SAM segmenter...")
    sam_segmenter = SAMCorrosionSegmenter()
    if sam_segmenter.predictor:
        print("✅ SAM2 segmenter initialized")
    else:
        print("⚠️ SAM2 segmenter not available (requires model weights)")
    
    # Initialize complete pipeline
    print("🔗 Initializing complete pipeline...")
    pipeline = RealTimeCorrosionPipeline()
    print("✅ Real-time pipeline initialized")
    
    # Initialize dashboard
    print("📊 Initializing dashboard...")
    dashboard = CorrosionDashboard()
    print("✅ Dashboard initialized")
    
    print("\n🎉 System initialization complete!")
    print("-" * 50)
    
    return {
        'rfdetr': rfdetr_model,
        'yolo': yolo_detector,
        'sam': sam_segmenter,
        'pipeline': pipeline,
        'dashboard': dashboard
    }

# Create demo test
print("🚀 Testing Corrosion Detection System")
print("="*60)

# Initialize system
system_components = initialize_system()

# Create demo data
print("\n📊 Creating demo data...")
demo_images, demo_annotations = create_demo_data()
print(f"✅ Created {len(demo_images)} demo images with annotations")

# Test with first demo image
test_image = demo_images[0]
print(f"\n🔍 Testing with demo image (shape: {test_image.shape})")

# Process through pipeline
results = system_components['pipeline'].process_frame(test_image)

print(f"\n📈 Processing Results:")
print(f"  • Detections found: {len(results['detections'])}")
print(f"  • Processing time: {results['performance']['processing_time']:.3f}s")
print(f"  • FPS: {results['performance']['fps']:.1f}")
print(f"  • Coverage: {results['analysis']['coverage_percentage']:.1f}%")
print(f"  • Risk level: {results['analysis']['risk_assessment']}")

# Visualize results
if results['detections']:
    print(f"\n🎨 Detected corrosion types:")
    for detection in results['detections']:
        print(f"  • {detection['class_name']} (confidence: {detection['confidence']:.2f})")
    
    # Show visualization
    visualize_detections(test_image, results['detections'], 
                        "Demo: Real-time Corrosion Detection")
else:
    print("ℹ️ No corrosion detected in demo image")

print(f"\n💡 System Status: Ready for deployment!")
print("="*60)

In [ ]:
# Training Example with Demo Data
def train_rfdetr_demo():
    """Demonstrate RF-DETR training with synthetic data"""
    
    print("🎓 RF-DETR Training Demonstration")
    print("="*50)
    
    # Create larger demo dataset
    print("📊 Generating training dataset...")
    demo_images, demo_annotations = create_demo_data()
    
    # Extend dataset for training demonstration
    extended_images = []
    extended_annotations = []
    
    for _ in range(50):  # Create 50 training samples
        demo_imgs, demo_anns = create_demo_data()
        extended_images.extend(demo_imgs)
        extended_annotations.extend(demo_anns)
    
    print(f"✅ Created {len(extended_images)} training samples")
    
    # Split data
    train_images = extended_images[:40]
    train_annotations = extended_annotations[:40] 
    val_images = extended_images[40:]
    val_annotations = extended_annotations[40:]
    
    print(f"📊 Train samples: {len(train_images)}, Val samples: {len(val_images)}")
    
    # Create datasets and dataloaders
    train_dataset = CorrosionDataset(
        image_paths=list(range(len(train_images))),  # Using indices as paths for demo
        annotations=train_annotations,
        transforms=get_train_transforms(),
        is_train=True
    )
    
    val_dataset = CorrosionDataset(
        image_paths=list(range(len(val_images))),
        annotations=val_annotations,
        transforms=get_val_transforms(),
        is_train=False
    )
    
    # Note: This is a simplified demo. In practice, you'd need proper DataLoaders
    # that handle the actual image loading from paths
    
    print("✅ Datasets created")
    
    # Initialize model and trainer
    print("🤖 Initializing RF-DETR model...")
    model = RFDETR(num_classes=6, num_queries=50)  # Smaller for demo
    
    # For demonstration purposes, we'll show the training setup
    # without actually running the full training loop
    
    print("🎯 Training Configuration:")
    print(f"  • Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"  • Training samples: {len(train_images)}")
    print(f"  • Validation samples: {len(val_images)}")
    print(f"  • Batch size: 4 (recommended)")
    print(f"  • Learning rate: 1e-4")
    print(f"  • Epochs: 100 (recommended)")
    
    # Simulate training metrics
    simulated_train_losses = np.linspace(2.5, 0.8, 20) + np.random.normal(0, 0.1, 20)
    simulated_val_losses = np.linspace(2.3, 1.0, 20) + np.random.normal(0, 0.15, 20)
    
    print("\n📈 Simulated training progress:")
    for epoch in range(0, 20, 5):
        print(f"  Epoch {epoch+1:2d}: Train Loss = {simulated_train_losses[epoch]:.3f}, "
              f"Val Loss = {simulated_val_losses[epoch]:.3f}")
    
    # Plot simulated training curves
    plot_training_history(simulated_train_losses, simulated_val_losses)
    
    print("\n✅ Training demonstration completed!")
    print("💡 To run actual training, uncomment the training loop in the trainer class")
    
    return model

# Run training demo
trained_model = train_rfdetr_demo()

In [ ]:
# Real-time Webcam Demo
def run_webcam_demo():
    """Run real-time corrosion detection on webcam feed"""
    
    print("📹 Starting Real-time Webcam Demo")
    print("="*50)
    
    # Initialize system
    pipeline = RealTimeCorrosionPipeline()
    dashboard = CorrosionDashboard()
    
    # Try to open webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Could not open webcam. Using demo mode instead.")
        run_demo_mode(pipeline, dashboard)
        return
    
    print("✅ Webcam opened successfully")
    print("🎮 Controls:")
    print("  • Press 'q' to quit")
    print("  • Press 's' to save screenshot")
    print("  • Press 'r' to reset statistics")
    
    frame_count = 0
    screenshot_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Failed to read frame from webcam")
                break
            
            # Resize frame for processing
            frame_resized = cv2.resize(frame, (640, 480))
            
            # Process frame
            results = pipeline.process_frame(frame_resized)
            
            # Update dashboard
            dashboard.update_performance_data(
                results['performance']['fps'],
                results['performance']['processing_time'],
                len(results['detections'])
            )
            dashboard.update_corrosion_analysis(results['analysis'])
            
            # Draw results on frame
            display_frame = results['annotated_frame'].copy()
            
            # Add performance info
            fps_text = f"FPS: {results['performance']['fps']:.1f}"
            cv2.putText(display_frame, fps_text, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            process_time_text = f"Process: {results['performance']['processing_time']*1000:.1f}ms"
            cv2.putText(display_frame, process_time_text, (10, 70), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            detections_text = f"Detections: {len(results['detections'])}"
            cv2.putText(display_frame, detections_text, (10, 110), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            coverage_text = f"Coverage: {results['analysis']['coverage_percentage']:.1f}%"
            cv2.putText(display_frame, coverage_text, (10, 150), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            risk_text = f"Risk: {results['analysis']['risk_assessment'].upper()}"
            risk_color = {'low': (0, 255, 0), 'medium': (0, 255, 255), 
                         'high': (0, 165, 255), 'critical': (0, 0, 255)}.get(
                         results['analysis']['risk_assessment'], (255, 255, 255))
            cv2.putText(display_frame, risk_text, (10, 190), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, risk_color, 2)
            
            # Show frame
            cv2.imshow('Real-time Corrosion Detection', display_frame)
            
            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                screenshot_path = f'screenshot_{screenshot_count:03d}.jpg'
                cv2.imwrite(screenshot_path, display_frame)
                print(f"📸 Screenshot saved: {screenshot_path}")
                screenshot_count += 1
            elif key == ord('r'):
                dashboard = CorrosionDashboard()
                print("🔄 Statistics reset")
            
            frame_count += 1
            
    except KeyboardInterrupt:
        print("\n⏹️ Demo stopped by user")
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        
        # Print final statistics
        stats = dashboard.get_current_stats()
        print(f"\n📊 Session Summary:")
        print(f"  • Frames processed: {frame_count}")
        print(f"  • Average FPS: {stats.get('current_fps', 0):.1f}")
        print(f"  • Screenshots saved: {screenshot_count}")
        print(f"  • Active alerts: {stats.get('active_alerts', 0)}")

def run_demo_mode(pipeline, dashboard):
    """Run demo mode with synthetic data"""
    
    print("🎮 Running Demo Mode")
    print("-" * 30)
    
    demo_images, _ = create_demo_data()
    
    for i, demo_image in enumerate(demo_images):
        print(f"Processing demo image {i+1}/{len(demo_images)}")
        
        # Process image
        results = pipeline.process_frame(demo_image)
        
        # Update dashboard
        dashboard.update_performance_data(
            results['performance']['fps'],
            results['performance']['processing_time'],
            len(results['detections'])
        )
        dashboard.update_corrosion_analysis(results['analysis'])
        
        # Show results
        if results['detections']:
            print(f"  ✅ Found {len(results['detections'])} corrosion areas")
            for det in results['detections']:
                print(f"    • {det['class_name']} (conf: {det['confidence']:.2f})")
        else:
            print(f"  ℹ️ No corrosion detected")
        
        time.sleep(1)  # Simulate processing time
    
    print("✅ Demo mode completed")

# Uncomment to run webcam demo
# run_webcam_demo()

In [ ]:
# Streamlit App Launcher
def launch_streamlit_app():
    """Launch the Streamlit dashboard application"""
    
    # Create the Streamlit app file
    app_code = '''
import streamlit as st
import cv2
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from PIL import Image

# Import our custom classes (assumes they're in the same environment)
# from corrosion_detection import RealTimeCorrosionPipeline, CorrosionDashboard

# Note: In a real deployment, you would import the actual classes
# For demo purposes, we'll create simplified versions

class SimplePipeline:
    def process_frame(self, frame):
        # Simulate processing
        time.sleep(0.1)  # Simulate processing time
        
        # Generate fake detections for demo
        detections = []
        if np.random.random() > 0.5:
            detections.append({
                'class_name': np.random.choice(['pitting_corrosion', 'uniform_corrosion', 'stress_corrosion']),
                'confidence': np.random.uniform(0.6, 0.95),
                'bbox': [100, 100, 200, 200]
            })
        
        annotated_frame = frame.copy()
        
        return {
            'annotated_frame': annotated_frame,
            'detections': detections,
            'analysis': {
                'total_corrosion_areas': len(detections),
                'coverage_percentage': np.random.uniform(0, 15),
                'risk_assessment': np.random.choice(['low', 'medium', 'high']),
                'corrosion_types': {'pitting_corrosion': 1} if detections else {},
                'recommendations': ['Monitor closely'] if detections else []
            },
            'performance': {
                'fps': np.random.uniform(20, 30),
                'processing_time': np.random.uniform(0.03, 0.08)
            }
        }

class SimpleDashboard:
    def __init__(self):
        self.performance_data = []
        self.corrosion_history = []
        self.alerts = []
    
    def update_performance_data(self, fps, processing_time, detections_count):
        timestamp = time.time()
        self.performance_data.append({
            'timestamp': timestamp,
            'fps': fps,
            'processing_time': processing_time,
            'detections_count': detections_count
        })
        if len(self.performance_data) > 50:
            self.performance_data.pop(0)
    
    def update_corrosion_analysis(self, analysis):
        timestamp = time.time()
        self.corrosion_history.append({
            'timestamp': timestamp,
            'coverage_percentage': analysis['coverage_percentage'],
            'risk_level': analysis['risk_assessment']
        })
        if len(self.corrosion_history) > 50:
            self.corrosion_history.pop(0)
    
    def get_current_stats(self):
        return {
            'current_fps': self.performance_data[-1]['fps'] if self.performance_data else 0,
            'current_coverage': self.corrosion_history[-1]['coverage_percentage'] if self.corrosion_history else 0,
            'current_risk': self.corrosion_history[-1]['risk_level'] if self.corrosion_history else 'low',
            'health_score': 75,
            'active_alerts': 0
        }

def main():
    st.set_page_config(
        page_title="Corrosion Detection Dashboard",
        page_icon="🔬",
        layout="wide"
    )
    
    st.title("🔬 Real-time Corrosion Detection System")
    st.markdown("### Advanced Computer Vision Pipeline for R&D Material Science Monitoring")
    
    # Initialize session state
    if 'pipeline' not in st.session_state:
        st.session_state.pipeline = SimplePipeline()
        st.session_state.dashboard = SimpleDashboard()
        st.session_state.is_running = False
    
    # Sidebar
    st.sidebar.header("⚙️ System Configuration")
    
    source_type = st.sidebar.selectbox(
        "Select Input Source",
        ["Demo Mode", "Image Upload"]
    )
    
    if source_type == "Image Upload":
        uploaded_file = st.sidebar.file_uploader("Upload Image", type=['jpg', 'jpeg', 'png'])
        
    # Control buttons
    col1, col2 = st.sidebar.columns(2)
    with col1:
        if st.button("▶️ Start"):
            st.session_state.is_running = True
    with col2:
        if st.button("⏸️ Stop"):
            st.session_state.is_running = False
    
    # Main content
    if st.session_state.is_running:
        col1, col2 = st.columns([2, 1])
        
        with col1:
            st.subheader("📹 Live Analysis")
            
            if source_type == "Image Upload" and 'uploaded_file' in locals() and uploaded_file:
                image = Image.open(uploaded_file)
                image_array = np.array(image)
                
                # Process image
                results = st.session_state.pipeline.process_frame(image_array)
                
                # Update dashboard
                st.session_state.dashboard.update_performance_data(
                    results['performance']['fps'],
                    results['performance']['processing_time'],
                    len(results['detections'])
                )
                st.session_state.dashboard.update_corrosion_analysis(results['analysis'])
                
                # Display results
                st.image(results['annotated_frame'], channels="RGB")
                
                if results['detections']:
                    st.success(f"Found {len(results['detections'])} corrosion areas")
                    for det in results['detections']:
                        st.write(f"• {det['class_name']} (confidence: {det['confidence']:.2f})")
                else:
                    st.info("No corrosion detected")
            
            else:  # Demo mode
                demo_image = np.random.randint(0, 255, (400, 600, 3), dtype=np.uint8)
                results = st.session_state.pipeline.process_frame(demo_image)
                
                st.session_state.dashboard.update_performance_data(
                    results['performance']['fps'],
                    results['performance']['processing_time'],
                    len(results['detections'])
                )
                st.session_state.dashboard.update_corrosion_analysis(results['analysis'])
                
                st.image(demo_image, channels="RGB", caption="Demo Mode - Synthetic Data")
        
        with col2:
            st.subheader("📊 System Status")
            
            stats = st.session_state.dashboard.get_current_stats()
            
            # Metrics
            col_a, col_b = st.columns(2)
            with col_a:
                st.metric("FPS", f"{stats['current_fps']:.1f}")
                st.metric("Coverage %", f"{stats['current_coverage']:.1f}%")
            with col_b:
                st.metric("Health Score", stats['health_score'])
                st.metric("Risk Level", stats['current_risk'].upper())
        
        # Performance charts
        st.subheader("📈 Performance Dashboard")
        
        if st.session_state.dashboard.performance_data:
            # Create performance plot
            df = pd.DataFrame(st.session_state.dashboard.performance_data)
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=df['timestamp'], y=df['fps'], name='FPS'))
            fig.update_layout(title='Real-time FPS', xaxis_title='Time', yaxis_title='FPS')
            st.plotly_chart(fig, use_container_width=True)
        
        # Auto-refresh every 2 seconds in demo mode
        if source_type == "Demo Mode":
            time.sleep(2)
            st.rerun()
    
    else:
        st.info("👆 Click 'Start' to begin monitoring")
        
        # Show features
        col1, col2, col3 = st.columns(3)
        with col1:
            st.markdown("""
            ### 🔧 Detection Pipeline
            - RF-DETR transformer model
            - YOLOv11 real-time detection
            - SAM2 precise segmentation
            """)
        with col2:
            st.markdown("""
            ### 📊 Analytics
            - Real-time performance metrics
            - Corrosion trend analysis
            - Risk assessment scoring
            """)
        with col3:
            st.markdown("""
            ### 🧪 R&D Benefits
            - Early detection capabilities
            - Automated monitoring
            - Data-driven insights
            """)

if __name__ == "__main__":
    main()
'''
    
    # Save the app file
    with open('streamlit_corrosion_app.py', 'w') as f:
        f.write(app_code)
    
    print("📱 Streamlit app created: streamlit_corrosion_app.py")
    print("🚀 To launch the dashboard, run:")
    print("   streamlit run streamlit_corrosion_app.py")
    print("\n🌐 The app will open in your web browser at http://localhost:8501")

# Create the Streamlit app
launch_streamlit_app()

In [ ]:
# Export and Deployment Functions
def export_model_for_production(model, model_name="rfdetr_corrosion"):
    """Export trained model for production deployment"""
    
    print(f"📦 Exporting {model_name} for production...")
    
    # Save PyTorch model
    torch.save(model.state_dict(), f"{model_name}.pth")
    print(f"✅ PyTorch model saved: {model_name}.pth")
    
    # Export to ONNX for cross-platform deployment
    try:
        dummy_input = torch.randn(1, 3, 512, 512).to(device)
        model.eval()
        
        torch.onnx.export(
            model,
            dummy_input,
            f"{model_name}.onnx",
            export_params=True,
            opset_version=11,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['class_logits', 'bbox_coords', 'severity_logits'],
            dynamic_axes={
                'input': {0: 'batch_size'},
                'class_logits': {0: 'batch_size'},
                'bbox_coords': {0: 'batch_size'},
                'severity_logits': {0: 'batch_size'}
            }
        )
        print(f"✅ ONNX model saved: {model_name}.onnx")
        
    except Exception as e:
        print(f"⚠️ ONNX export failed: {e}")
    
    # Create model info file
    model_info = {
        'model_name': model_name,
        'model_type': 'RF-DETR',
        'num_classes': 6,
        'input_size': [512, 512],
        'classes': [
            'uniform_corrosion', 'pitting_corrosion', 'crevice_corrosion',
            'stress_corrosion', 'galvanic_corrosion', 'erosion_corrosion'
        ],
        'severity_levels': ['none', 'mild', 'moderate', 'severe', 'critical'],
        'preprocessing': {
            'normalize': True,
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225]
        },
        'postprocessing': {
            'confidence_threshold': 0.5,
            'nms_threshold': 0.4
        }
    }
    
    import json
    with open(f"{model_name}_info.json", 'w') as f:
        json.dump(model_info, f, indent=2)
    
    print(f"✅ Model info saved: {model_name}_info.json")
    print(f"📦 Export completed for {model_name}")

def create_deployment_package():
    """Create deployment package with all necessary files"""
    
    print("📦 Creating deployment package...")
    
    # Create requirements.txt
    requirements = """
torch>=1.9.0
torchvision>=0.10.0
opencv-python>=4.5.0
numpy>=1.21.0
pillow>=8.3.0
matplotlib>=3.4.0
seaborn>=0.11.0
pandas>=1.3.0
scikit-learn>=1.0.0
ultralytics>=8.0.0
streamlit>=1.28.0
plotly>=5.0.0
supervision>=0.16.0
albumentations>=1.3.0
segment-anything>=1.0
scipy>=1.7.0
""".strip()
    
    with open('requirements.txt', 'w') as f:
        f.write(requirements)
    
    # Create deployment script
    deploy_script = '''
#!/usr/bin/env python3
"""
Corrosion Detection System Deployment Script
"""

import os
import sys
import argparse
import torch
import cv2
import numpy as np
from pathlib import Path

def setup_environment():
    """Setup the deployment environment"""
    print("🔧 Setting up deployment environment...")
    
    # Install requirements
    os.system(f"{sys.executable} -m pip install -r requirements.txt")
    
    print("✅ Environment setup completed")

def deploy_models():
    """Deploy models for inference"""
    print("🚀 Deploying models...")
    
    # Check for model files
    required_files = [
        'rfdetr_corrosion.pth',
        'rfdetr_corrosion_info.json'
    ]
    
    missing_files = [f for f in required_files if not os.path.exists(f)]
    if missing_files:
        print(f"❌ Missing model files: {missing_files}")
        return False
    
    print("✅ All model files found")
    return True

def run_inference_server(port=8000):
    """Run inference server"""
    print(f"🌐 Starting inference server on port {port}...")
    
    # This would start your inference server
    # For now, we'll just show the command
    print(f"📝 Run: python inference_server.py --port {port}")

def main():
    parser = argparse.ArgumentParser(description='Deploy Corrosion Detection System')
    parser.add_argument('--setup', action='store_true', help='Setup environment')
    parser.add_argument('--deploy', action='store_true', help='Deploy models')
    parser.add_argument('--serve', action='store_true', help='Start inference server')
    parser.add_argument('--port', type=int, default=8000, help='Server port')
    
    args = parser.parse_args()
    
    if args.setup:
        setup_environment()
    
    if args.deploy:
        deploy_models()
    
    if args.serve:
        run_inference_server(args.port)
    
    if not any([args.setup, args.deploy, args.serve]):
        print("🔬 Corrosion Detection System")
        print("Usage: python deploy.py [--setup] [--deploy] [--serve]")

if __name__ == "__main__":
    main()
'''.strip()
    
    with open('deploy.py', 'w') as f:
        f.write(deploy_script)
    
    # Create Docker configuration
    dockerfile = '''
FROM python:3.9-slim

WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \\
    libglib2.0-0 \\
    libsm6 \\
    libxext6 \\
    libxrender-dev \\
    libgomp1 \\
    libglib2.0-0 \\
    && rm -rf /var/lib/apt/lists/*

# Copy requirements and install Python dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application files
COPY . .

# Expose port
EXPOSE 8501

# Run Streamlit app
CMD ["streamlit", "run", "streamlit_corrosion_app.py", "--server.port=8501", "--server.address=0.0.0.0"]
'''.strip()
    
    with open('Dockerfile', 'w') as f:
        f.write(dockerfile)
    
    # Create docker-compose.yml
    docker_compose = '''
version: '3.8'

services:
  corrosion-detection:
    build: .
    ports:
      - "8501:8501"
    volumes:
      - "./models:/app/models"
      - "./data:/app/data"
    environment:
      - PYTHONUNBUFFERED=1
    restart: unless-stopped

  inference-api:
    build: .
    ports:
      - "8000:8000" 
    volumes:
      - "./models:/app/models"
    command: ["python", "inference_server.py", "--port", "8000"]
    restart: unless-stopped
'''.strip()
    
    with open('docker-compose.yml', 'w') as f:
        f.write(docker_compose)
    
    print("✅ Requirements.txt created")
    print("✅ Deployment script created: deploy.py")
    print("✅ Dockerfile created")
    print("✅ Docker Compose configuration created")
    print("\n🚀 Deployment package ready!")
    print("📋 Next steps:")
    print("  1. Train your models and export them")
    print("  2. Run: python deploy.py --setup")
    print("  3. Run: python deploy.py --deploy")
    print("  4. Run: docker-compose up (for containerized deployment)")

# Create deployment package
create_deployment_package()

In [ ]:
# Performance Benchmarking and Optimization
class PerformanceBenchmark:
    """Benchmark system performance and optimization metrics"""
    
    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.benchmark_results = {}
    
    def benchmark_inference_speed(self, test_images, num_runs=10):
        """Benchmark inference speed across different image sizes"""
        
        print("⏱️ Benchmarking Inference Speed...")
        print("-" * 40)
        
        image_sizes = [(320, 240), (640, 480), (1024, 768), (1920, 1080)]
        results = {}
        
        for size in image_sizes:
            print(f"Testing size {size[0]}x{size[1]}...")
            times = []
            
            for _ in range(num_runs):
                # Create test image
                test_image = cv2.resize(test_images[0], size)
                
                # Measure inference time
                start_time = time.time()
                _ = self.pipeline.process_frame(test_image)
                end_time = time.time()
                
                times.append(end_time - start_time)
            
            avg_time = np.mean(times)
            std_time = np.std(times)
            fps = 1.0 / avg_time
            
            results[f"{size[0]}x{size[1]}"] = {
                'avg_time': avg_time,
                'std_time': std_time,
                'fps': fps,
                'min_time': np.min(times),
                'max_time': np.max(times)
            }
            
            print(f"  Average: {avg_time:.3f}s ± {std_time:.3f}s ({fps:.1f} FPS)")
        
        self.benchmark_results['inference_speed'] = results
        return results
    
    def benchmark_memory_usage(self, test_image):
        """Benchmark memory usage during inference"""
        
        import psutil
        import gc
        
        print("🧠 Benchmarking Memory Usage...")
        print("-" * 30)
        
        # Get initial memory
        process = psutil.Process()
        initial_memory = process.memory_info().rss / 1024 / 1024  # MB
        
        # Run inference and measure peak memory
        gc.collect()
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
        start_memory = process.memory_info().rss / 1024 / 1024
        
        # Process multiple frames to get peak usage
        for _ in range(10):
            _ = self.pipeline.process_frame(test_image)
        
        peak_memory = process.memory_info().rss / 1024 / 1024
        
        # GPU memory if available
        gpu_memory = 0
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.max_memory_allocated() / 1024 / 1024  # MB
        
        memory_results = {
            'initial_memory_mb': initial_memory,
            'start_memory_mb': start_memory,
            'peak_memory_mb': peak_memory,
            'memory_increase_mb': peak_memory - start_memory,
            'gpu_memory_mb': gpu_memory
        }
        
        print(f"Initial memory: {initial_memory:.1f} MB")
        print(f"Peak memory: {peak_memory:.1f} MB")
        print(f"Memory increase: {memory_results['memory_increase_mb']:.1f} MB")
        if gpu_memory > 0:
            print(f"GPU memory: {gpu_memory:.1f} MB")
        
        self.benchmark_results['memory_usage'] = memory_results
        return memory_results
    
    def benchmark_accuracy_vs_speed(self, test_dataset, confidence_thresholds):
        """Benchmark accuracy vs speed tradeoff"""
        
        print("🎯 Benchmarking Accuracy vs Speed...")
        print("-" * 35)
        
        results = {}
        
        for threshold in confidence_thresholds:
            print(f"Testing confidence threshold: {threshold}")
            
            # Update threshold
            self.pipeline.yolo_detector.conf_threshold = threshold
            
            # Measure speed and accuracy
            inference_times = []
            detection_counts = []
            
            for test_image, _ in test_dataset[:10]:  # Test on subset
                start_time = time.time()
                result = self.pipeline.process_frame(test_image)
                end_time = time.time()
                
                inference_times.append(end_time - start_time)
                detection_counts.append(len(result['detections']))
            
            avg_time = np.mean(inference_times)
            avg_detections = np.mean(detection_counts)
            fps = 1.0 / avg_time
            
            results[threshold] = {
                'avg_inference_time': avg_time,
                'fps': fps,
                'avg_detections': avg_detections,
                'total_detections': np.sum(detection_counts)
            }
            
            print(f"  FPS: {fps:.1f}, Avg detections: {avg_detections:.1f}")
        
        self.benchmark_results['accuracy_vs_speed'] = results
        return results
    
    def generate_benchmark_report(self):
        """Generate comprehensive benchmark report"""
        
        print("\\n" + "="*60)
        print("PERFORMANCE BENCHMARK REPORT")
        print("="*60)
        
        # Inference Speed Report
        if 'inference_speed' in self.benchmark_results:
            print("\\n📊 INFERENCE SPEED ANALYSIS")
            print("-" * 30)
            
            for size, metrics in self.benchmark_results['inference_speed'].items():
                print(f"Resolution {size}:")
                print(f"  • Average Time: {metrics['avg_time']:.3f}s")
                print(f"  • FPS: {metrics['fps']:.1f}")
                print(f"  • Time Range: {metrics['min_time']:.3f}s - {metrics['max_time']:.3f}s")
        
        # Memory Usage Report
        if 'memory_usage' in self.benchmark_results:
            print("\\n🧠 MEMORY USAGE ANALYSIS")
            print("-" * 25)
            
            mem = self.benchmark_results['memory_usage']
            print(f"  • Initial Memory: {mem['initial_memory_mb']:.1f} MB")
            print(f"  • Peak Memory: {mem['peak_memory_mb']:.1f} MB")
            print(f"  • Memory Increase: {mem['memory_increase_mb']:.1f} MB")
            if mem['gpu_memory_mb'] > 0:
                print(f"  • GPU Memory: {mem['gpu_memory_mb']:.1f} MB")
        
        # Accuracy vs Speed Report  
        if 'accuracy_vs_speed' in self.benchmark_results:
            print("\\n⚖️ ACCURACY vs SPEED TRADEOFF")
            print("-" * 32)
            
            for threshold, metrics in self.benchmark_results['accuracy_vs_speed'].items():
                print(f"Confidence {threshold}:")
                print(f"  • FPS: {metrics['fps']:.1f}")
                print(f"  • Avg Detections: {metrics['avg_detections']:.1f}")
        
        # Optimization Recommendations
        print("\\n🚀 OPTIMIZATION RECOMMENDATIONS")
        print("-" * 35)
        
        if 'inference_speed' in self.benchmark_results:
            fps_values = [m['fps'] for m in self.benchmark_results['inference_speed'].values()]
            max_fps = max(fps_values)
            
            if max_fps < 15:
                print("  • Consider using a smaller model variant (YOLOv11n)")
                print("  • Reduce input resolution for better performance")
                print("  • Implement model quantization")
            elif max_fps < 25:
                print("  • Good performance, consider GPU acceleration if not used")
                print("  • Optimize preprocessing pipeline")
            else:
                print("  • Excellent performance achieved!")
                print("  • Consider increasing model complexity for better accuracy")
        
        print("\\n" + "="*60)

# Run performance benchmark
def run_performance_benchmark():
    """Run comprehensive performance benchmark"""
    
    print("🏃 Starting Performance Benchmark...")
    
    # Initialize system
    pipeline = RealTimeCorrosionPipeline()
    benchmark = PerformanceBenchmark(pipeline)
    
    # Create test data
    test_images, _ = create_demo_data()
    
    # Run benchmarks
    benchmark.benchmark_inference_speed(test_images, num_runs=5)
    benchmark.benchmark_memory_usage(test_images[0])
    
    # Test different confidence thresholds
    confidence_thresholds = [0.3, 0.5, 0.7, 0.9]
    benchmark.benchmark_accuracy_vs_speed(test_images, confidence_thresholds)
    
    # Generate report
    benchmark.generate_benchmark_report()
    
    return benchmark

# Uncomment to run benchmark
# benchmark = run_performance_benchmark()

In [ ]:
# Advanced Analytics and Reporting
class CorrosionAnalytics:
    """Advanced analytics for corrosion monitoring and reporting"""
    
    def __init__(self):
        self.detection_history = []
        self.material_database = {}
        self.alert_history = []
        
    def add_detection_record(self, timestamp, detections, image_metadata=None):
        """Add detection record to history"""
        record = {
            'timestamp': timestamp,
            'detections': detections,
            'metadata': image_metadata or {},
            'total_areas': len(detections),
            'severity_scores': []
        }
        
        # Calculate severity scores
        for detection in detections:
            severity_score = self.calculate_severity_score(detection)
            record['severity_scores'].append(severity_score)
        
        self.detection_history.append(record)
    
    def calculate_severity_score(self, detection):
        """Calculate severity score based on corrosion type and characteristics"""
        base_scores = {
            'uniform_corrosion': 3,
            'pitting_corrosion': 7,
            'crevice_corrosion': 6,
            'stress_corrosion': 9,
            'galvanic_corrosion': 5,
            'erosion_corrosion': 8
        }
        
        base_score = base_scores.get(detection['class_name'], 5)
        confidence_factor = detection['confidence']
        
        # Adjust based on detection area (if available)
        area_factor = 1.0
        if 'area' in detection:
            # Larger areas are more severe
            area_factor = min(1.5, 1.0 + detection['area'] / 10000)
        
        severity_score = base_score * confidence_factor * area_factor
        return min(10, severity_score)  # Cap at 10
    
    def generate_trend_analysis(self, days=30):
        """Generate trend analysis for corrosion progression"""
        
        if not self.detection_history:
            return None
        
        # Filter records by time period
        cutoff_time = time.time() - (days * 24 * 3600)
        recent_records = [r for r in self.detection_history if r['timestamp'] >= cutoff_time]
        
        if not recent_records:
            return None
        
        # Calculate trends
        timestamps = [r['timestamp'] for r in recent_records]
        total_detections = [r['total_areas'] for r in recent_records]
        avg_severity = [np.mean(r['severity_scores']) if r['severity_scores'] else 0 
                       for r in recent_records]
        
        # Linear regression for trends
        x = np.array(range(len(timestamps)))
        
        detection_trend = np.polyfit(x, total_detections, 1)[0] if len(x) > 1 else 0
        severity_trend = np.polyfit(x, avg_severity, 1)[0] if len(x) > 1 else 0
        
        # Corrosion type distribution
        type_counts = {}
        for record in recent_records:
            for detection in record['detections']:
                ctype = detection['class_name']
                type_counts[ctype] = type_counts.get(ctype, 0) + 1
        
        trend_analysis = {
            'period_days': days,
            'total_records': len(recent_records),
            'detection_trend': detection_trend,  # per day
            'severity_trend': severity_trend,    # per day
            'avg_detections_per_day': len(total_detections) / days,
            'current_avg_severity': np.mean(avg_severity[-7:]) if len(avg_severity) >= 7 else 0,
            'type_distribution': type_counts,
            'trend_direction': 'increasing' if detection_trend > 0.1 else 'decreasing' if detection_trend < -0.1 else 'stable'
        }
        
        return trend_analysis
    
    def predict_maintenance_needs(self, trend_analysis):
        """Predict maintenance needs based on trend analysis"""
        
        if not trend_analysis:
            return {'priority': 'low', 'recommendations': ['Continue monitoring']}
        
        predictions = {
            'priority': 'low',
            'urgency_score': 0,
            'predicted_failure_days': None,
            'recommendations': [],
            'risk_factors': []
        }
        
        urgency_score = 0
        
        # Analyze detection trends
        if trend_analysis['detection_trend'] > 0.5:
            urgency_score += 3
            predictions['risk_factors'].append('Rapidly increasing corrosion rate')
        elif trend_analysis['detection_trend'] > 0.1:
            urgency_score += 1
            predictions['risk_factors'].append('Gradual increase in corrosion')
        
        # Analyze severity trends
        if trend_analysis['severity_trend'] > 0.2:
            urgency_score += 2
            predictions['risk_factors'].append('Increasing corrosion severity')
        
        # Analyze current severity
        if trend_analysis['current_avg_severity'] > 8:
            urgency_score += 3
            predictions['risk_factors'].append('High current severity levels')
        elif trend_analysis['current_avg_severity'] > 6:
            urgency_score += 2
            predictions['risk_factors'].append('Moderate current severity levels')
        
        # Check for critical corrosion types
        critical_types = ['stress_corrosion', 'erosion_corrosion']
        for ctype in critical_types:
            if ctype in trend_analysis['type_distribution'] and trend_analysis['type_distribution'][ctype] > 2:
                urgency_score += 2
                predictions['risk_factors'].append(f'Multiple {ctype.replace("_", " ")} instances detected')
        
        # Determine priority and recommendations
        if urgency_score >= 8:
            predictions['priority'] = 'critical'
            predictions['predicted_failure_days'] = 7
            predictions['recommendations'] = [
                'Immediate inspection and intervention required',
                'Consider emergency maintenance procedures',
                'Implement temporary protective measures'
            ]
        elif urgency_score >= 5:
            predictions['priority'] = 'high'
            predictions['predicted_failure_days'] = 30
            predictions['recommendations'] = [
                'Schedule maintenance within 1-2 weeks',
                'Increase monitoring frequency',
                'Prepare maintenance materials and resources'
            ]
        elif urgency_score >= 3:
            predictions['priority'] = 'medium'
            predictions['predicted_failure_days'] = 90
            predictions['recommendations'] = [
                'Schedule maintenance within 1-2 months',
                'Continue regular monitoring',
                'Consider preventive treatments'
            ]
        else:
            predictions['priority'] = 'low'
            predictions['recommendations'] = [
                'Continue regular monitoring',
                'Follow standard maintenance schedule',
                'Document current condition for baseline'
            ]
        
        predictions['urgency_score'] = urgency_score
        return predictions
    
    def generate_comprehensive_report(self, material_id=None):
        """Generate comprehensive corrosion analysis report"""
        
        print("\\n" + "="*80)
        print("COMPREHENSIVE CORROSION ANALYSIS REPORT")
        print("="*80)
        
        if material_id:
            print(f"Material ID: {material_id}")
        
        print(f"Report Generated: {pd.Timestamp.now()}")
        print(f"Total Detection Records: {len(self.detection_history)}")
        
        # Current Status
        if self.detection_history:
            latest_record = self.detection_history[-1]
            print(f"\\n🔍 CURRENT STATUS")
            print("-" * 20)
            print(f"Last Inspection: {pd.to_datetime(latest_record['timestamp'], unit='s')}")
            print(f"Active Corrosion Areas: {latest_record['total_areas']}")
            if latest_record['severity_scores']:
                avg_severity = np.mean(latest_record['severity_scores'])
                max_severity = np.max(latest_record['severity_scores'])
                print(f"Average Severity: {avg_severity:.1f}/10")
                print(f"Maximum Severity: {max_severity:.1f}/10")
        
        # Trend Analysis
        trend_30d = self.generate_trend_analysis(30)
        if trend_30d:
            print(f"\\n📈 30-DAY TREND ANALYSIS")
            print("-" * 25)
            print(f"Trend Direction: {trend_30d['trend_direction'].upper()}")
            print(f"Detection Rate Change: {trend_30d['detection_trend']:+.2f} areas/day")
            print(f"Severity Trend: {trend_30d['severity_trend']:+.2f} points/day")
            
            print(f"\\n🧬 CORROSION TYPE DISTRIBUTION")
            print("-" * 30)
            for ctype, count in sorted(trend_30d['type_distribution'].items(), 
                                     key=lambda x: x[1], reverse=True):
                percentage = (count / sum(trend_30d['type_distribution'].values())) * 100
                print(f"  • {ctype.replace('_', ' ').title()}: {count} ({percentage:.1f}%)")
        
        # Maintenance Predictions
        predictions = self.predict_maintenance_needs(trend_30d)
        print(f"\\n🔧 MAINTENANCE PREDICTIONS")
        print("-" * 25)
        print(f"Priority Level: {predictions['priority'].upper()}")
        print(f"Urgency Score: {predictions['urgency_score']}/10")
        
        if predictions['predicted_failure_days']:
            print(f"Predicted Action Needed Within: {predictions['predicted_failure_days']} days")
        
        if predictions['risk_factors']:
            print(f"\\n⚠️ RISK FACTORS:")
            for factor in predictions['risk_factors']:
                print(f"  • {factor}")
        
        print(f"\\n💡 RECOMMENDATIONS:")
        for rec in predictions['recommendations']:
            print(f"  • {rec}")
        
        print("\\n" + "="*80)
        
        return {
            'current_status': latest_record if self.detection_history else None,
            'trend_analysis': trend_30d,
            'predictions': predictions,
            'report_timestamp': time.time()
        }

# Demo analytics usage
def demo_analytics():
    """Demonstrate analytics capabilities"""
    
    print("📊 Demonstrating Advanced Analytics...")
    
    analytics = CorrosionAnalytics()
    
    # Simulate detection history over 30 days
    base_time = time.time() - (30 * 24 * 3600)
    
    for day in range(30):
        for session in range(np.random.randint(1, 4)):  # 1-3 sessions per day
            timestamp = base_time + day * 24 * 3600 + session * 8 * 3600
            
            # Generate random detections with increasing trend
            num_detections = max(0, np.random.poisson(2 + day * 0.1))
            detections = []
            
            for _ in range(num_detections):
                detection = {
                    'class_name': np.random.choice([
                        'uniform_corrosion', 'pitting_corrosion', 'stress_corrosion'
                    ]),
                    'confidence': np.random.uniform(0.6, 0.95),
                    'area': np.random.uniform(100, 1000)
                }
                detections.append(detection)
            
            analytics.add_detection_record(timestamp, detections)
    
    # Generate comprehensive report
    report = analytics.generate_comprehensive_report("DEMO_MATERIAL_001")
    
    return analytics, report

# Run analytics demo
analytics_demo, demo_report = demo_analytics()

In [ ]:
# Integration with Laboratory Equipment
class LabEquipmentInterface:
    """Interface for integrating with laboratory equipment and sensors"""
    
    def __init__(self):
        self.connected_devices = {}
        self.sensor_readings = {}
        self.calibration_data = {}
    
    def connect_microscope(self, microscope_id, connection_params):
        """Connect to digital microscope for high-resolution imaging"""
        
        print(f"🔬 Connecting to microscope {microscope_id}...")
        
        # Simulate microscope connection
        self.connected_devices[microscope_id] = {
            'type': 'digital_microscope',
            'status': 'connected',
            'max_resolution': connection_params.get('resolution', '4K'),
            'magnification_range': connection_params.get('magnification', '10x-1000x'),
            'last_calibration': time.time()
        }
        
        print(f"✅ Microscope {microscope_id} connected successfully")
        return True
    
    def connect_environmental_sensors(self, sensor_ids):
        """Connect to environmental monitoring sensors"""
        
        for sensor_id in sensor_ids:
            print(f"🌡️ Connecting to environmental sensor {sensor_id}...")
            
            self.connected_devices[sensor_id] = {
                'type': 'environmental_sensor',
                'status': 'connected',
                'parameters': ['temperature', 'humidity', 'pH', 'conductivity'],
                'last_reading': time.time()
            }
        
        print(f"✅ Connected {len(sensor_ids)} environmental sensors")
    
    def capture_high_res_image(self, microscope_id, magnification='100x'):
        """Capture high-resolution image from microscope"""
        
        if microscope_id not in self.connected_devices:
            raise ValueError(f"Microscope {microscope_id} not connected")
        
        # Simulate high-resolution image capture
        print(f"📸 Capturing image at {magnification} magnification...")
        
        # Generate synthetic high-res corrosion image
        image_size = (2048, 2048) if magnification.endswith('x') and int(magnification[:-1]) > 50 else (1024, 1024)
        image = np.random.randint(80, 120, (*image_size, 3), dtype=np.uint8)
        
        # Add realistic corrosion features
        center_x, center_y = image_size[0] // 2, image_size[1] // 2
        
        # Add corrosion spot
        corrosion_radius = np.random.randint(50, 200)
        y, x = np.ogrid[:image_size[0], :image_size[1]]
        mask = (x - center_x)**2 + (y - center_y)**2 <= corrosion_radius**2
        
        # Apply corrosion coloring
        image[mask] = [139, 69, 19]  # Brown corrosion color
        
        # Add noise for realism
        noise = np.random.normal(0, 10, image.shape).astype(np.int16)
        image = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        
        metadata = {
            'microscope_id': microscope_id,
            'magnification': magnification,
            'resolution': image_size,
            'timestamp': time.time(),
            'exposure_time': '1/60s',
            'lighting': 'LED white light'
        }
        
        return image, metadata
    
    def read_environmental_data(self):
        """Read data from all connected environmental sensors"""
        
        readings = {}
        
        for device_id, device_info in self.connected_devices.items():
            if device_info['type'] == 'environmental_sensor':
                
                # Simulate sensor readings
                reading = {
                    'temperature': np.random.normal(23, 2),  # °C
                    'humidity': np.random.normal(55, 10),    # %
                    'pH': np.random.normal(7.2, 0.5),       # pH units
                    'conductivity': np.random.normal(500, 50), # μS/cm
                    'timestamp': time.time()
                }
                
                readings[device_id] = reading
                self.sensor_readings[device_id] = reading
        
        return readings
    
    def correlate_corrosion_with_environment(self, corrosion_data, environmental_data):
        """Correlate corrosion detection with environmental conditions"""
        
        print("🔗 Correlating corrosion with environmental conditions...")
        
        correlations = {}
        
        if not environmental_data:
            return correlations
        
        # Calculate average environmental conditions
        avg_temp = np.mean([data['temperature'] for data in environmental_data.values()])
        avg_humidity = np.mean([data['humidity'] for data in environmental_data.values()])
        avg_ph = np.mean([data['pH'] for data in environmental_data.values()])
        avg_conductivity = np.mean([data['conductivity'] for data in environmental_data.values()])
        
        # Determine risk factors based on environmental conditions
        risk_factors = []
        
        if avg_temp > 30:
            risk_factors.append('High temperature accelerates corrosion')
        if avg_humidity > 70:
            risk_factors.append('High humidity promotes corrosion')
        if avg_ph < 6.5 or avg_ph > 8.5:
            risk_factors.append('Non-neutral pH increases corrosion risk')
        if avg_conductivity > 800:
            risk_factors.append('High conductivity indicates corrosive environment')
        
        # Calculate corrosion severity based on environmental factors
        severity_multiplier = 1.0
        
        if avg_temp > 25:
            severity_multiplier *= 1 + (avg_temp - 25) * 0.02
        if avg_humidity > 60:
            severity_multiplier *= 1 + (avg_humidity - 60) * 0.01
        if avg_ph < 7:
            severity_multiplier *= 1 + (7 - avg_ph) * 0.1
        elif avg_ph > 7.5:
            severity_multiplier *= 1 + (avg_ph - 7.5) * 0.05
        if avg_conductivity > 500:
            severity_multiplier *= 1 + (avg_conductivity - 500) * 0.0005
        
        correlations = {
            'environmental_conditions': {
                'temperature': avg_temp,
                'humidity': avg_humidity,
                'pH': avg_ph,
                'conductivity': avg_conductivity
            },
            'risk_factors': risk_factors,
            'severity_multiplier': severity_multiplier,
            'corrosion_acceleration_factor': severity_multiplier,
            'recommendations': self._generate_environmental_recommendations(
                avg_temp, avg_humidity, avg_ph, avg_conductivity
            )
        }
        
        return correlations
    
    def _generate_environmental_recommendations(self, temp, humidity, ph, conductivity):
        """Generate recommendations based on environmental conditions"""
        
        recommendations = []
        
        if temp > 30:
            recommendations.append('Implement cooling system to reduce temperature')
        if humidity > 70:
            recommendations.append('Install dehumidification system')
        if ph < 6.5:
            recommendations.append('Apply alkaline treatment to neutralize acidity')
        elif ph > 8.5:
            recommendations.append('Apply acid treatment to reduce alkalinity')
        if conductivity > 800:
            recommendations.append('Reduce ionic concentration in environment')
        
        if not recommendations:
            recommendations.append('Environmental conditions are within acceptable range')
        
        return recommendations

# Laboratory Data Management
class LabDataManager:
    """Manage laboratory data, samples, and experimental records"""
    
    def __init__(self):
        self.sample_database = {}
        self.experiment_logs = []
        self.analysis_history = {}
    
    def register_sample(self, sample_id, material_info, preparation_details):
        """Register a new sample for analysis"""
        
        sample_record = {
            'sample_id': sample_id,
            'registration_date': time.time(),
            'material_type': material_info.get('type', 'unknown'),
            'composition': material_info.get('composition', {}),
            'dimensions': material_info.get('dimensions', {}),
            'preparation': preparation_details,
            'analysis_history': [],
            'status': 'active'
        }
        
        self.sample_database[sample_id] = sample_record
        
        print(f"📝 Sample {sample_id} registered successfully")
        return sample_record
    
    def log_analysis_session(self, sample_id, analysis_results, equipment_used):
        """Log an analysis session for a sample"""
        
        if sample_id not in self.sample_database:
            raise ValueError(f"Sample {sample_id} not found in database")
        
        session_log = {
            'session_id': f"{sample_id}_{int(time.time())}",
            'timestamp': time.time(),
            'sample_id': sample_id,
            'equipment_used': equipment_used,
            'analysis_results': analysis_results,
            'analyst': 'AI_System',
            'session_duration': analysis_results.get('processing_time', 0)
        }
        
        self.experiment_logs.append(session_log)
        self.sample_database[sample_id]['analysis_history'].append(session_log['session_id'])
        
        print(f"📊 Analysis session logged for sample {sample_id}")
        return session_log
    
    def export_lab_report(self, sample_id, format='PDF'):
        """Export comprehensive laboratory report"""
        
        if sample_id not in self.sample_database:
            raise ValueError(f"Sample {sample_id} not found")
        
        sample = self.sample_database[sample_id]
        
        # Gather all analysis sessions for this sample
        sample_sessions = [log for log in self.experiment_logs 
                          if log['sample_id'] == sample_id]
        
        report = {
            'report_title': f'Corrosion Analysis Report - Sample {sample_id}',
            'report_date': pd.Timestamp.now().isoformat(),
            'sample_information': sample,
            'analysis_sessions': sample_sessions,
            'summary': self._generate_sample_summary(sample_sessions),
            'format': format
        }
        
        # Save report
        import json
        filename = f"lab_report_{sample_id}_{int(time.time())}.json"
        with open(filename, 'w') as f:
            json.dump(report, f, indent=2, default=str)
        
        print(f"📋 Lab report exported: {filename}")
        return report
    
    def _generate_sample_summary(self, sessions):
        """Generate summary statistics for sample analysis"""
        
        if not sessions:
            return {'total_sessions': 0, 'analysis_summary': 'No analysis performed'}
        
        total_detections = []
        severity_scores = []
        
        for session in sessions:
            results = session['analysis_results']
            if 'detections' in results:
                total_detections.append(len(results['detections']))
            if 'analysis' in results and 'coverage_percentage' in results['analysis']:
                severity_scores.append(results['analysis']['coverage_percentage'])
        
        summary = {
            'total_sessions': len(sessions),
            'first_analysis': min(session['timestamp'] for session in sessions),
            'last_analysis': max(session['timestamp'] for session in sessions),
            'avg_detections_per_session': np.mean(total_detections) if total_detections else 0,
            'max_coverage_observed': max(severity_scores) if severity_scores else 0,
            'corrosion_progression': 'increasing' if len(severity_scores) > 1 and 
                                   severity_scores[-1] > severity_scores[0] else 'stable'
        }
        
        return summary

# Demo laboratory integration
def demo_lab_integration():
    """Demonstrate laboratory equipment integration"""
    
    print("🧪 Demonstrating Laboratory Integration...")
    print("=" * 50)
    
    # Initialize lab interface and data manager
    lab_interface = LabEquipmentInterface()
    data_manager = LabDataManager()
    
    # Connect equipment
    lab_interface.connect_microscope('MICROSCOPE_001', {
        'resolution': '4K',
        'magnification': '10x-1000x'
    })
    
    lab_interface.connect_environmental_sensors([
        'ENV_SENSOR_001', 'ENV_SENSOR_002'
    ])
    
    # Register a sample
    sample_info = {
        'type': 'carbon_steel',
        'composition': {'Fe': 0.98, 'C': 0.02},
        'dimensions': {'length': 50, 'width': 25, 'thickness': 5}  # mm
    }
    
    preparation_details = {
        'surface_preparation': 'polished',
        'exposure_conditions': 'salt spray 24h',
        'preparation_date': time.time() - 86400  # 24h ago
    }
    
    sample_record = data_manager.register_sample(
        'SAMPLE_001', sample_info, preparation_details
    )
    
    # Capture high-resolution image
    image, metadata = lab_interface.capture_high_res_image('MICROSCOPE_001', '200x')
    
    # Read environmental data
    env_data = lab_interface.read_environmental_data()
    
    # Process image through corrosion detection
    pipeline = RealTimeCorrosionPipeline()
    analysis_results = pipeline.process_frame(image)
    
    # Add metadata to results
    analysis_results['image_metadata'] = metadata
    analysis_results['environmental_data'] = env_data
    
    # Correlate with environmental conditions
    correlation = lab_interface.correlate_corrosion_with_environment(
        analysis_results, env_data
    )
    analysis_results['environmental_correlation'] = correlation
    
    # Log analysis session
    session_log = data_manager.log_analysis_session(
        'SAMPLE_001',
        analysis_results,
        ['MICROSCOPE_001', 'ENV_SENSOR_001', 'ENV_SENSOR_002']
    )
    
    # Export lab report
    report = data_manager.export_lab_report('SAMPLE_001')
    
    print("\\n✅ Laboratory integration demo completed!")
    print(f"📸 Captured image: {metadata['resolution']} at {metadata['magnification']}")
    print(f"🌡️ Environmental readings: {len(env_data)} sensors")
    print(f"🔍 Detected {len(analysis_results['detections'])} corrosion areas")
    print(f"📋 Lab report generated with {len(report['analysis_sessions'])} sessions")
    
    return lab_interface, data_manager, report

# Run laboratory integration demo
lab_demo_results = demo_lab_integration()

In [ ]:
# API Service for Remote Access
class CorrosionDetectionAPI:
    """REST API service for remote corrosion detection access"""
    
    def __init__(self, pipeline, analytics):
        self.pipeline = pipeline
        self.analytics = analytics
        self.api_keys = {}
        self.usage_logs = []
    
    def generate_api_key(self, user_id, permissions=['read']):
        """Generate API key for user access"""
        import secrets
        api_key = secrets.token_urlsafe(32)
        
        self.api_keys[api_key] = {
            'user_id': user_id,
            'permissions': permissions,
            'created_at': time.time(),
            'last_used': None,
            'usage_count': 0
        }
        
        return api_key
    
    def validate_api_key(self, api_key, required_permission='read'):
        """Validate API key and permissions"""
        
        if api_key not in self.api_keys:
            return False, "Invalid API key"
        
        key_info = self.api_keys[api_key]
        
        if required_permission not in key_info['permissions']:
            return False, f"Permission '{required_permission}' not granted"
        
        # Update usage
        key_info['last_used'] = time.time()
        key_info['usage_count'] += 1
        
        return True, "Valid API key"
    
    def process_image_endpoint(self, image_data, api_key, analysis_options=None):
        """API endpoint for image processing"""
        
        # Validate API key
        valid, message = self.validate_api_key(api_key, 'read')
        if not valid:
            return {'error': message, 'status': 401}
        
        try:
            # Decode image data (assuming base64 encoded)
            import base64
            from io import BytesIO
            
            if isinstance(image_data, str):
                image_bytes = base64.b64decode(image_data)
                image = cv2.imdecode(
                    np.frombuffer(image_bytes, np.uint8), 
                    cv2.IMREAD_COLOR
                )
            else:
                image = image_data
            
            # Process image
            results = self.pipeline.process_frame(image)
            
            # Log API usage
            self.usage_logs.append({
                'timestamp': time.time(),
                'api_key': api_key,
                'endpoint': 'process_image',
                'success': True,
                'detections_found': len(results['detections'])
            })
            
            # Prepare response
            response = {
                'status': 200,
                'timestamp': time.time(),
                'detections': results['detections'],
                'analysis': results['analysis'],
                'performance': results['performance']
            }
            
            # Add additional analysis if requested
            if analysis_options:
                if analysis_options.get('include_trends', False):
                    trends = self.analytics.generate_trend_analysis(7)  # 7-day trends
                    response['trend_analysis'] = trends
                
                if analysis_options.get('include_predictions', False):
                    trends = self.analytics.generate_trend_analysis(7)
                    predictions = self.analytics.predict_maintenance_needs(trends)
                    response['predictions'] = predictions
            
            return response
            
        except Exception as e:
            # Log error
            self.usage_logs.append({
                'timestamp': time.time(),
                'api_key': api_key,
                'endpoint': 'process_image',
                'success': False,
                'error': str(e)
            })
            
            return {'error': f'Processing failed: {str(e)}', 'status': 500}
    
    def get_system_status_endpoint(self, api_key):
        """API endpoint for system status"""
        
        valid, message = self.validate_api_key(api_key, 'read')
        if not valid:
            return {'error': message, 'status': 401}
        
        # System status information
        status = {
            'status': 200,
            'timestamp': time.time(),
            'system_status': 'operational',
            'models_loaded': {
                'yolo': self.pipeline.yolo_detector is not None,
                'sam': self.pipeline.sam_segmenter.predictor is not None,
                'rfdetr': self.pipeline.rfdetr is not None
            },
            'performance_stats': {
                'avg_processing_time': np.mean(self.pipeline.processing_times[-10:]) if self.pipeline.processing_times else 0,
                'current_fps': len(self.pipeline.processing_times) / (time.time() - self.pipeline.start_time) if self.pipeline.processing_times else 0
            },
            'api_usage': {
                'total_requests': len(self.usage_logs),
                'successful_requests': len([log for log in self.usage_logs if log['success']]),
                'failed_requests': len([log for log in self.usage_logs if not log['success']])
            }
        }
        
        return status
    
    def get_analytics_endpoint(self, api_key, time_range='7d'):
        """API endpoint for analytics data"""
        
        valid, message = self.validate_api_key(api_key, 'read')
        if not valid:
            return {'error': message, 'status': 401}
        
        # Parse time range
        if time_range == '7d':
            days = 7
        elif time_range == '30d':
            days = 30
        elif time_range == '90d':
            days = 90
        else:
            days = 7
        
        # Generate analytics
        trend_analysis = self.analytics.generate_trend_analysis(days)
        predictions = self.analytics.predict_maintenance_needs(trend_analysis)
        
        response = {
            'status': 200,
            'timestamp': time.time(),
            'time_range': f'{days} days',
            'trend_analysis': trend_analysis,
            'predictions': predictions
        }
        
        return response

# Create Flask Web Service
def create_flask_service():
    """Create Flask web service for API deployment"""
    
    service_code = '''
from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
import numpy as np
import cv2
import time

# Import our detection system
# from corrosion_detection import RealTimeCorrosionPipeline, CorrosionAnalytics, CorrosionDetectionAPI

app = Flask(__name__)
CORS(app)

# Initialize system (in production, load from saved models)
# pipeline = RealTimeCorrosionPipeline()
# analytics = CorrosionAnalytics()
# api_service = CorrosionDetectionAPI(pipeline, analytics)

# For demo purposes, create a simple mock API
class MockAPI:
    def __init__(self):
        self.api_keys = {'demo_key_123': {'user_id': 'demo_user', 'permissions': ['read']}}
    
    def validate_api_key(self, api_key, permission='read'):
        return api_key in self.api_keys, "Valid" if api_key in self.api_keys else "Invalid"
    
    def process_image_endpoint(self, image_data, api_key, analysis_options=None):
        valid, msg = self.validate_api_key(api_key)
        if not valid:
            return {'error': 'Invalid API key', 'status': 401}
        
        return {
            'status': 200,
            'timestamp': time.time(),
            'detections': [
                {'class_name': 'pitting_corrosion', 'confidence': 0.85, 'bbox': [100, 100, 200, 200]}
            ],
            'analysis': {
                'total_corrosion_areas': 1,
                'coverage_percentage': 5.2,
                'risk_assessment': 'medium'
            },
            'performance': {'fps': 25.3, 'processing_time': 0.039}
        }

api_service = MockAPI()

@app.route('/api/v1/detect', methods=['POST'])
def detect_corrosion():
    """Process image for corrosion detection"""
    
    try:
        # Get API key from headers
        api_key = request.headers.get('Authorization', '').replace('Bearer ', '')
        if not api_key:
            return jsonify({'error': 'API key required'}), 401
        
        # Get image data
        data = request.get_json()
        if 'image' not in data:
            return jsonify({'error': 'Image data required'}), 400
        
        # Get analysis options
        analysis_options = data.get('options', {})
        
        # Process image
        result = api_service.process_image_endpoint(
            data['image'], api_key, analysis_options
        )
        
        return jsonify(result), result.get('status', 200)
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/v1/status', methods=['GET'])
def get_status():
    """Get system status"""
    
    api_key = request.headers.get('Authorization', '').replace('Bearer ', '')
    if not api_key:
        return jsonify({'error': 'API key required'}), 401
    
    # Mock status response
    status = {
        'status': 200,
        'timestamp': time.time(),
        'system_status': 'operational',
        'models_loaded': {'yolo': True, 'sam': True, 'rfdetr': True},
        'version': '1.0.0'
    }
    
    return jsonify(status)

@app.route('/api/v1/analytics', methods=['GET'])
def get_analytics():
    """Get analytics data"""
    
    api_key = request.headers.get('Authorization', '').replace('Bearer ', '')
    if not api_key:
        return jsonify({'error': 'API key required'}), 401
    
    time_range = request.args.get('range', '7d')
    
    # Mock analytics response
    analytics = {
        'status': 200,
        'timestamp': time.time(),
        'time_range': time_range,
        'trend_analysis': {
            'detection_trend': 0.1,
            'severity_trend': 0.05,
            'trend_direction': 'stable'
        },
        'predictions': {
            'priority': 'medium',
            'recommendations': ['Continue monitoring', 'Schedule inspection']
        }
    }
    
    return jsonify(analytics)

@app.route('/api/v1/docs', methods=['GET'])
def api_documentation():
    """API documentation"""
    
    docs = {
        'title': 'Corrosion Detection API',
        'version': '1.0.0',
        'endpoints': {
            'POST /api/v1/detect': {
                'description': 'Process image for corrosion detection',
                'headers': {'Authorization': 'Bearer YOUR_API_KEY'},
                'body': {
                    'image': 'base64 encoded image data',
                    'options': {
                        'include_trends': 'boolean',
                        'include_predictions': 'boolean'
                    }
                }
            },
            'GET /api/v1/status': {
                'description': 'Get system status',
                'headers': {'Authorization': 'Bearer YOUR_API_KEY'}
            },
            'GET /api/v1/analytics?range=7d': {
                'description': 'Get analytics data',
                'headers': {'Authorization': 'Bearer YOUR_API_KEY'},
                'parameters': {'range': '7d, 30d, or 90d'}
            }
        },
        'authentication': {
            'type': 'API Key',
            'header': 'Authorization: Bearer YOUR_API_KEY',
            'demo_key': 'demo_key_123'
        }
    }
    
    return jsonify(docs)

@app.route('/')
def home():
    """API home page"""
    return jsonify({
        'message': 'Corrosion Detection API',
        'version': '1.0.0',
        'documentation': '/api/v1/docs',
        'status': 'operational'
    })

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
'''.strip()
    
    # Save Flask service
    with open('flask_api_service.py', 'w') as f:
        f.write(service_code)
    
    print("🌐 Flask API service created: flask_api_service.py")
    print("🚀 To start the API server, run:")
    print("   python flask_api_service.py")
    print("📖 API documentation will be available at: http://localhost:5000/api/v1/docs")
    print("🔑 Demo API key: demo_key_123")

# Create the Flask service
create_flask_service()

# Demo API usage
def demo_api_usage():
    """Demonstrate API usage with example requests"""
    
    print("\\n📡 API Usage Examples:")
    print("=" * 30)
    
    # Example curl commands
    examples = '''
# 1. Get API documentation
curl -X GET http://localhost:5000/api/v1/docs

# 2. Check system status
curl -X GET http://localhost:5000/api/v1/status \\
  -H "Authorization: Bearer demo_key_123"

# 3. Process image for corrosion detection
curl -X POST http://localhost:5000/api/v1/detect \\
  -H "Authorization: Bearer demo_key_123" \\
  -H "Content-Type: application/json" \\
  -d '{
    "image": "base64_encoded_image_data_here",
    "options": {
      "include_trends": true,
      "include_predictions": true
    }
  }'

# 4. Get analytics data
curl -X GET "http://localhost:5000/api/v1/analytics?range=7d" \\
  -H "Authorization: Bearer demo_key_123"
'''.strip()
    
    print(examples)
    
    # Python client example
    python_client = '''
import requests
import base64

# API configuration
API_BASE_URL = "http://localhost:5000/api/v1"
API_KEY = "demo_key_123"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

# Example 1: Check system status
response = requests.get(f"{API_BASE_URL}/status", headers=HEADERS)
print("Status:", response.json())

# Example 2: Process image
with open("image.jpg", "rb") as f:
    image_data = base64.b64encode(f.read()).decode()

detection_request = {
    "image": image_data,
    "options": {"include_trends": True, "include_predictions": True}
}

response = requests.post(
    f"{API_BASE_URL}/detect", 
    json=detection_request, 
    headers={**HEADERS, "Content-Type": "application/json"}
)
print("Detection results:", response.json())

# Example 3: Get analytics
response = requests.get(f"{API_BASE_URL}/analytics?range=30d", headers=HEADERS)
print("Analytics:", response.json())
'''.strip()
    
    print("\\n🐍 Python Client Example:")
    print("-" * 25)
    print(python_client)
    
    # Save Python client example
    with open('api_client_example.py', 'w') as f:
        f.write(python_client)
    
    print("\\n💾 Python client example saved: api_client_example.py")

demo_api_usage()